A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
classdef Swaption < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        tenor
        strike
        annuityIn
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        callStrike
        
    end
    
    methods
        % constructor
        function rangeAccrual = Swaption(params)
                security = Security(params);
                rangeAccrual.payCcy = security.payCcy;
                rangeAccrual.startDate = security.startDate;
                rangeAccrual.endDate = security.endDate;
                rangeAccrual.maturity = security.maturity;
                
%                 rangeAccrual.payCcy = params.payCcy;
%                 rangeAccrual.startDate = params.startDate;
%                 rangeAccrual.maturity = params.maturity;
                
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.tenor = params.params('tenor');
                rangeAccrual.strike = params.params('strike');
                rangeAccrual.annuityIn = params.params('annuityIn');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,1);
                
                end
                 
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                
                ralowerS = rangeAccrual.lowerS;
                raupperS = rangeAccrual.upperS;
                
                tenorCMS = rangeAccrual.tenor;
                
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxCMS = zeros(1,pathSize);
                
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                
                idxC = zeros(1,pathSize);
                idxN = zeros(1,pathSize);
                
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                    idxC = idxCMS;
                    
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
%                     probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    
                    idxN = idxC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef SwapRateHelper_Old < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        fixedLegTenor
        fixedLegConvention
        fixedLegDayCounter
        
        iborIndex
        evaluationDate
        
        vanillaSwap
        discountCurve
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function swapRateHelper = SwapRateHelper_Old(params)
                if nargin > 0
                    swapRateHelper.rateType = 'Swap';
                    
                    swapRateHelper.rate = params('rate');
                    swapRateHelper.rateTenor = params('rateTenor');
                    
                    swapRateHelper.fixedLegTenor = params('fixedLegTenor');
                    swapRateHelper.fixedLegConvention = params('fixedLegConvention');
                    swapRateHelper.fixedLegDayCounter = params('fixedLegDayCounter');
                                        
                    iborIndex = params('IborIndex');
                    swapRateHelper.iborIndex = iborIndex;
                    
                    swapRateHelper.evaluationDate = iborIndex.forwardCurve.asOfDate;
                    
                    swapRateHelper.discountCurve = params('discountCurve');
                    
                    if swapRateHelper.discountCurve.asOfDate ~= iborIndex.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    swapRateHelper.vanillaSwap = MakeVanillaSwap(swapRateHelper);
                    
                else
                    swapRateHelper.rate = 0.02;
                    swapRateHelper.iborIndex = IborIndex();
                end
                
                out = InitializeDates(swapRateHelper);
                
                swapRateHelper.earliestDate = out.earliestDate;
                swapRateHelper.latestDate = out.latestDate;
                
        end
        
        function out = MakeVanillaSwap(swapRateHelper)
            iborIndex = swapRateHelper.iborIndex;
            referenceDate = iborIndex.forwardCurve.asOfDate;
            startDate = iborIndex.fixingCalendar.Advance(referenceDate,iborIndex.convention,...
                                                iborIndex.fixingDays,'day',iborIndex.endOfMonth);
            
            endDate = AddTenor(startDate,swapRateHelper.rateTenor);
            % fixed & floating schedule 
            fixedSchedule = Schedule(startDate,endDate,swapRateHelper.fixedLegTenor ,iborIndex.fixingCalendar,...
                                    swapRateHelper.fixedLegConvention, 'Backward',false); 
            
            floatingSchedule = Schedule(startDate,endDate,iborIndex.tenor,iborIndex.fixingCalendar,...
                                    iborIndex.convention, 'Backward',false);
             
             type = 'Payer';
%              nominal = 10000.0;
             nominal = 1.0;
             fixedRate = 0.0;
             fixedDayCounter = swapRateHelper.fixedLegDayCounter;
             floatingSpread = 0.0;
             floatingDayCounter = iborIndex.dayCounter;
             vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
                                      floatingSchedule,iborIndex,floatingSpread,floatingDayCounter);
                                  
             out = vanillaSwap;
        end
        
        function out = InitializeDates(swapRateHelper)
            
            %earliestDate
            earliestDate = swapRateHelper.vanillaSwap.StartDate();
            latestDate = swapRateHelper.vanillaSwap.EndDate();
            
            floatEndValueDate = swapRateHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
            % if the last floating Index fixingEndDate is after
            % latestDate(last accrualEndDate then we change
            % the lastestDate into floatEndValueDate
            
            if DateDiff(floatEndValueDate,latestDate) > 0
                latestDate = floatEndValueDate;
            end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(swapRateHelper)
            outInfo = swapRateHelper.vanillaSwap.Calculate(swapRateHelper.discountCurve);
            out = outInfo.fairRate;
            
        end
        
        % bootstrapping
        function out = QuoteError(swapRateHelper)
            out = swapRateHelper.rate - swapRateHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(swapRateHelper,guess,idx)
            swapRateHelper.Update(idx,guess);
            out = swapRateHelper.QuoteError();
        end
        
        function Update(swapRateHelper,idx,guess)
             % forwardCurve update
            forwardRawData = swapRateHelper.iborIndex.forwardCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            swapRateHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % discountCurve update
            discountRawData = swapRateHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            swapRateHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef SwapRateHelper_DS < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        fixedLegTenor
        fixedLegConvention
        fixedLegDayCounter
        
        floatingLegTenor
        floatingLegConvention
        floatingLegDayCounter
        
        iborIndex
        evaluationDate
        
        vanillaSwap
        discountCurve
        
        earliestDate
        latestDate
        
        spotLag
        payLag
        
        calcCalendar
        
    end
    
    methods
        % constructor
        function swapRateHelper = SwapRateHelper_DS(params)
                if nargin > 0
                    swapRateHelper.rateType = 'Swap';
                    
                    swapRateHelper.calcCalendar = params('calcCalendar');
                    
                    if isKey(params,'spotLag')
                        swapRateHelper.spotLag = params('spotLag');
                    else
                        swapRateHelper.spotLag = 2; % default
                     end
                    
                    if isKey(params,'payLag')
                        swapRateHelper.payLag = params('payLag');
                    else
                        swapRateHelper.payLag = 0; % default
                    end
                    
                    
                    swapRateHelper.rate = params('rate');
                    swapRateHelper.rateTenor = params('rateTenor');
                    
                    swapRateHelper.fixedLegTenor = params('fixedLegTenor');
                    swapRateHelper.fixedLegConvention = params('fixedLegConvention');
                    swapRateHelper.fixedLegDayCounter = params('fixedLegDayCounter');
                    
                    swapRateHelper.floatingLegTenor = params('floatingLegTenor');
                    swapRateHelper.floatingLegConvention = params('floatingLegConvention');
                    swapRateHelper.floatingLegDayCounter = params('floatingLegDayCounter');
                    
                                        
                    iborIndex = params('IborIndex');
                    swapRateHelper.iborIndex = iborIndex;
                    
                    swapRateHelper.evaluationDate = iborIndex.forwardCurve.asOfDate;
                    
                    swapRateHelper.discountCurve = params('discountCurve');
                    
                    if DateDiff(swapRateHelper.discountCurve.asOfDate,iborIndex.forwardCurve.asOfDate)~=0
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    
                    swapRateHelper.vanillaSwap = MakeVanillaSwap(swapRateHelper);
                    
                else
                    swapRateHelper.rate = 0.02;
                    swapRateHelper.iborIndex = IborIndex();
                end
                
                out = InitializeDates(swapRateHelper);
                
                swapRateHelper.earliestDate = out.earliestDate;
                swapRateHelper.latestDate = out.latestDate;
                
        end
        
        function out = MakeVanillaSwap(swapRateHelper)
            iborIndex = swapRateHelper.iborIndex;
            referenceDate = iborIndex.forwardCurve.asOfDate;
            
            startDate = swapRateHelper.calcCalendar.Advance(referenceDate,iborIndex.convention,...
                                                swapRateHelper.spotLag,'day',iborIndex.endOfMonth);
                                            
            useEndOfMonth = false;
            if IsEndOfMonth(startDate)
               useEndOfMonth = true;
            end
            
            endDate = AddTenor(startDate,swapRateHelper.rateTenor);
            
            % fixed & floating schedule 
%             fixedSchedule = Schedule(startDate,endDate,swapRateHelper.fixedLegTenor ,iborIndex.fixingCalendar,...
%                                     swapRateHelper.fixedLegConvention, 'Backward',false); 
%             
%             floatingSchedule = Schedule(startDate,endDate,iborIndex.tenor,iborIndex.fixingCalendar,...
%                                     iborIndex.convention, 'Backward',false);
            
            fixedSchedule = ScheduleWTermConvention(startDate,endDate,swapRateHelper.fixedLegTenor ,swapRateHelper.calcCalendar,...
                                    swapRateHelper.fixedLegConvention,swapRateHelper.fixedLegConvention, 'Backward',useEndOfMonth); 
            
            floatingSchedule = ScheduleWTermConvention(startDate,endDate,swapRateHelper.floatingLegTenor,swapRateHelper.calcCalendar,...
                                    swapRateHelper.floatingLegConvention,swapRateHelper.fixedLegConvention, 'Backward',useEndOfMonth);

                                
             type = 'Payer';

             nominal = 1.0;
             fixedRate = 0.0;
             fixedDayCounter = swapRateHelper.fixedLegDayCounter;
             floatingSpread = 0.0;
             floatingDayCounter = iborIndex.dayCounter;
             payLag = swapRateHelper.payLag;
             vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
                                      floatingSchedule,iborIndex,floatingSpread,floatingDayCounter,payLag);
                                  
             out = vanillaSwap;
        end
        
        function out = InitializeDates(swapRateHelper)
            
            %earliestDate
            earliestDate = swapRateHelper.vanillaSwap.StartDate();
            latestDate = swapRateHelper.vanillaSwap.EndDate();
            
            % use PillarDate As last paydate only when
            % useLastEndDateAsPillarDate is false
            if ~swapRateHelper.iborIndex.useLastEndDateAsPillarDate
            
                latestReleventDate = swapRateHelper.vanillaSwap.floatingLeg.couponRates{end}.paymentDate; 
                % if the last floating Index fixingEndDate is after
                % latestDate(last accrualEndDate then we change
                % the lastestDate into floatEndValueDate

                if DateDiff(latestReleventDate,latestDate) > 0
                    latestDate = latestReleventDate;
                end
            end
            
%             floatEndValueDate = swapRateHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(swapRateHelper)
            outInfo = swapRateHelper.vanillaSwap.Calculate(swapRateHelper.discountCurve);
            out = outInfo.fairRate;
            
        end
        
        % bootstrapping
        function out = QuoteError(swapRateHelper)
            out = swapRateHelper.rate - swapRateHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(swapRateHelper,guess,idx)
            swapRateHelper.Update(idx,guess);
            out = swapRateHelper.QuoteError();
        end
        
        function Update(swapRateHelper,idx,guess)
             % forwardCurve update
            forwardRawData = swapRateHelper.iborIndex.forwardCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            swapRateHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            % Multicurve, Dual Curve Stripping we do not update discount
            % Curve
            % disabled
            
%             % discountCurve update
%             discountRawData = swapRateHelper.discountCurve.params('rawData');
%             discountRawData(idx,2) = guess;
%             swapRateHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef SwapRateHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        fixedLegTenor
        fixedLegConvention
        fixedLegDayCounter
        
        floatingLegTenor
        floatingLegConvention
        floatingLegDayCounter
        
        iborIndex
        evaluationDate
        
        vanillaSwap
        discountCurve
        
        earliestDate
        latestDate
        
        spotLag
        payLag
        
        calcCalendar
        
    end
    
    methods
        % constructor
        function swapRateHelper = SwapRateHelper(params)
                if nargin > 0
                    swapRateHelper.rateType = 'Swap';
                    
                    swapRateHelper.calcCalendar = params('calcCalendar');
                    
                    if isKey(params,'spotLag')
                        swapRateHelper.spotLag = params('spotLag');
                    else
                        swapRateHelper.spotLag = 2; % default
                     end
                    
                    if isKey(params,'payLag')
                        swapRateHelper.payLag = params('payLag');
                    else
                        swapRateHelper.payLag = 0; % default
                    end
                    
                    
                    swapRateHelper.rate = params('rate');
                    swapRateHelper.rateTenor = params('rateTenor');
                    
                    swapRateHelper.fixedLegTenor = params('fixedLegTenor');
                    swapRateHelper.fixedLegConvention = params('fixedLegConvention');
                    swapRateHelper.fixedLegDayCounter = params('fixedLegDayCounter');
                    
                    swapRateHelper.floatingLegTenor = params('floatingLegTenor');
                    swapRateHelper.floatingLegConvention = params('floatingLegConvention');
                    swapRateHelper.floatingLegDayCounter = params('floatingLegDayCounter');
                    
                                        
                    iborIndex = params('IborIndex');
                    swapRateHelper.iborIndex = iborIndex;
                    
                    swapRateHelper.evaluationDate = iborIndex.forwardCurve.asOfDate;
                    
                    swapRateHelper.discountCurve = params('discountCurve');
                    
                    if DateDiff(swapRateHelper.discountCurve.asOfDate,iborIndex.forwardCurve.asOfDate)~=0
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    
                    swapRateHelper.vanillaSwap = MakeVanillaSwap(swapRateHelper);
                    
                else
                    swapRateHelper.rate = 0.02;
                    swapRateHelper.iborIndex = IborIndex();
                end
                
                out = InitializeDates(swapRateHelper);
                
                swapRateHelper.earliestDate = out.earliestDate;
                swapRateHelper.latestDate = out.latestDate;
                
        end
        
        function out = MakeVanillaSwap(swapRateHelper)
            iborIndex = swapRateHelper.iborIndex;
            referenceDate = iborIndex.forwardCurve.asOfDate;
            
            startDate = swapRateHelper.calcCalendar.Advance(referenceDate,iborIndex.convention,...
                                                swapRateHelper.spotLag,'day',iborIndex.endOfMonth);
                                            
            useEndOfMonth = false;
            if IsEndOfMonth(startDate)
               useEndOfMonth = true;
            end
            
            endDate = AddTenor(startDate,swapRateHelper.rateTenor);
            
            % fixed & floating schedule 
%             fixedSchedule = Schedule(startDate,endDate,swapRateHelper.fixedLegTenor ,iborIndex.fixingCalendar,...
%                                     swapRateHelper.fixedLegConvention, 'Backward',false); 
%             
%             floatingSchedule = Schedule(startDate,endDate,iborIndex.tenor,iborIndex.fixingCalendar,...
%                                     iborIndex.convention, 'Backward',false);
            
            fixedSchedule = ScheduleWTermConvention(startDate,endDate,swapRateHelper.fixedLegTenor ,swapRateHelper.calcCalendar,...
                                    swapRateHelper.fixedLegConvention,swapRateHelper.fixedLegConvention, 'Backward',useEndOfMonth); 
            
            floatingSchedule = ScheduleWTermConvention(startDate,endDate,swapRateHelper.floatingLegTenor,swapRateHelper.calcCalendar,...
                                    swapRateHelper.floatingLegConvention,swapRateHelper.fixedLegConvention, 'Backward',useEndOfMonth);

                                
             type = 'Payer';

             nominal = 1.0;
             fixedRate = 0.0;
             fixedDayCounter = swapRateHelper.fixedLegDayCounter;
             floatingSpread = 0.0;
             floatingDayCounter = iborIndex.dayCounter;
             payLag = swapRateHelper.payLag;
             vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
                                      floatingSchedule,iborIndex,floatingSpread,floatingDayCounter,payLag);
                                  
             out = vanillaSwap;
        end
        
        function out = InitializeDates(swapRateHelper)
            
            %earliestDate
            earliestDate = swapRateHelper.vanillaSwap.StartDate();
            latestDate = swapRateHelper.vanillaSwap.EndDate();
            
            % use PillarDate As last paydate only when
            % useLastEndDateAsPillarDate is false
            if ~swapRateHelper.iborIndex.useLastEndDateAsPillarDate
            
                latestReleventDate = swapRateHelper.vanillaSwap.floatingLeg.couponRates{end}.paymentDate; 
                % if the last floating Index fixingEndDate is after
                % latestDate(last accrualEndDate then we change
                % the lastestDate into floatEndValueDate

                if DateDiff(latestReleventDate,latestDate) > 0
                    latestDate = latestReleventDate;
                end
            end
            
%             floatEndValueDate = swapRateHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(swapRateHelper)
            outInfo = swapRateHelper.vanillaSwap.Calculate(swapRateHelper.discountCurve);
            out = outInfo.fairRate;
            
        end
        
        % bootstrapping
        function out = QuoteError(swapRateHelper)
            out = swapRateHelper.rate - swapRateHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(swapRateHelper,guess,idx)
            swapRateHelper.Update(idx,guess);
            out = swapRateHelper.QuoteError();
        end
        
        function Update(swapRateHelper,idx,guess)
             % forwardCurve update
            forwardRawData = swapRateHelper.iborIndex.forwardCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            swapRateHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            % Multicurve, Dual Curve Stripping we do not update discount
            % Curve
            % disabled
            
            % discountCurve update
            discountRawData = swapRateHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            swapRateHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef SwapPointHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        rateType
        
        rate
        rateTenor
        
        exchangeRate
        OriginalDomesticAmount
        
        calendar
        convention
        fixingDays
        
        evaluationDate
        
        foreignDCurve
%         DomesticDCurve
        
        swapPoint
        discountCurve
        
        earliestDate
        latestDate
        effectiveDate
        maturityDate
    end
    
    methods
        % constructor
        function swapPointHelper = SwapPointHelper(params)
                if nargin > 0
                    swapPointHelper.rateType = 'SwapPoint';
                    
                    swapPointHelper.rate = params('rate');
                    swapPointHelper.rateTenor = params('rateTenor');
                    swapPointHelper.fixingDays = params('fixingDays');
                    
                    swapPointHelper.exchangeRate = params('exchangeRate');
                    swapPointHelper.OriginalDomesticAmount = swapPointHelper.exchangeRate;
                    swapPointHelper.foreignDCurve = params('foreignDCurve');
                    swapPointHelper.calendar = params('calendar');
                    swapPointHelper.convention = params('convention');
                    swapPointHelper.evaluationDate = swapPointHelper.foreignDCurve.asOfDate;
                end
                
                out = InitializeDates(swapPointHelper);
                
                swapPointHelper.earliestDate = out.earliestDate;
                swapPointHelper.latestDate = out.latestDate;
                swapPointHelper.maturityDate = out.maturityDate;
                swapPointHelper.effectiveDate = out.effectiveDate;
                
                %for bootstrapping 
                tempSwapPoint = 0.0;
%                 swapPointHelper.swapPoint = SwapPoint("Long",swapPointHelper.exchangeRate,1.0,tempSwapPoint,...
%                                                       0.0, swapPointHelper.effectiveDate,swapPointHelper.maturityDate,swapPointHelper.foreignDCurve);
                                                  
                swapPointHelper.swapPoint = SwapPoint("Long",swapPointHelper.exchangeRate,1.0,tempSwapPoint,...
                                                      0.0, swapPointHelper.effectiveDate,swapPointHelper.maturityDate); 
        end
        
        function out = InitializeDates(swapPointHelper)
            
            %earliestDate
            earliestDate = swapPointHelper.evaluationDate;
            effectiveDate = swapPointHelper.calendar.Advance(earliestDate,swapPointHelper.convention,...
                                                                swapPointHelper.fixingDays,'day',false);
            
            latestDate_temp = AddTenor(effectiveDate,swapPointHelper.rateTenor);
            latestDate = swapPointHelper.calendar.Adjust(latestDate_temp,swapPointHelper.convention);
            
%             latestDate = swapPointHelper.calendar.AdvanceTenor(effectiveDate,swapPointHelper.convention,...
%             swapPointHelper.rateTenor,false);
                                                           
%             % latestDate or maturityDate                                               
%             if strcmp(swapPointHelper.rateTenor(end),'D')
%                 latestDate = swapPointHelper.calendar.Advance(effectiveDate,swapPointHelper.convention,...
%                                                                 str2num(swapPointHelper.rateTenor(1:end-1)),'day',false);
%             
%             elseif strcmp(swapPointHelper.rateTenor(end),'M')
%                 latestDate = swapPointHelper.calendar.Advance(effectiveDate,swapPointHelper.convention,...
%                                                                 str2num(swapPointHelper.rateTenor(1:end-1)),'month',false);
%             else
%                 error('unimplemented')
%             end
            
%             fixingDate = iborIndex.fixingCalendar.Advance(earliestDate,iborIndex.convention,...
%                                                                 -1*iborIndex.fixingDays,'day',iborIndex.endOfMonth);
%                                 
           out.earliestDate = earliestDate;
           out.effectiveDate = effectiveDate;
           out.latestDate = latestDate;
           out.maturityDate = latestDate;
           
        end
        
        % core function
        
        function out = ImpliedQuote(swapPointHelper)
            outInfo = swapPointHelper.swapPoint.Calculate(swapPointHelper.discountCurve,swapPointHelper.foreignDCurve);
            out = outInfo.fairSwapPoint;
        end
        
        % bootstrapping
        function out = QuoteError(swapPointHelper)
            out = swapPointHelper.rate - swapPointHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(swapPointHelper,guess,idx)
            swapPointHelper.Update(idx,guess);
            out = swapPointHelper.QuoteError();
        end
        
        function Update(swapPointHelper,idx,guess)
             % domesticCurve update
            discountRawData = swapPointHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            swapPointHelper.discountCurve.params('rawData')= discountRawData;
            
        end
    end
    
end



In [ ]:
classdef SwapPoint < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        domesticAmount
        foreignAmount
        swapPointValue
        bumpingSpread
%         foreignDCurve
        
        effectiveDate
        maturityDate
        
        domesticLeg
        foreignLeg
        
        NPV;
        fairRate;
       
    end
    
    methods
        % constructor
        function swapPoint = SwapPoint(type,domesticAmount,foreignAmount,swapPointValue,bumpingSpread, ...
                                      effectiveDate,maturityDate)
            
            swapPoint.type = type;
            swapPoint.domesticAmount = domesticAmount;
            swapPoint.foreignAmount = foreignAmount;
            swapPoint.swapPointValue = swapPointValue;
            swapPoint.bumpingSpread = bumpingSpread;
%             swapPoint.foreignDCurve = foreignDCurve;
            
            swapPoint.effectiveDate = effectiveDate;
            swapPoint.maturityDate = maturityDate;
            
            simpleCashflow = SimpleCashflow(-domesticAmount,effectiveDate);
            
            swapPoint.domesticLeg = Leg({simpleCashflow});
            
            simpleCashflow = SimpleCashflow(domesticAmount + swapPointValue,maturityDate);
            
            swapPoint.domesticLeg.cashflows{end+1} = simpleCashflow;
            
            simpleCashflow = SimpleCashflow(-foreignAmount,effectiveDate);
            swapPoint.foreignLeg = Leg({simpleCashflow});
            simpleCashflow = SimpleCashflow(foreignAmount,maturityDate);
            swapPoint.foreignLeg.cashflows{end+1} = simpleCashflow;
            
            
        end
        
%         function out = StartDate(vanillaSwap)
%             d1 = vanillaSwap.fixedLeg.StartDate();
%             d2 = vanillaSwap.floatingLeg.StartDate();
%             
%             if DateDiff(d1,d2) > 0
%                 out = d2;
%             else
%                 out = d1;
%             end
%         end
%         
%         function out = EndDate(vanillaSwap)
%             d1 = vanillaSwap.fixedLeg.EndDate();
%             d2 = vanillaSwap.floatingLeg.EndDate();
%             
%             if DateDiff(d1,d2) > 0
%                 out = d1;
%             else
%                 out = d2;
%             end
%         end
%         
        function out = Calculate(swapPoint,discountCurve,foreignDCurve)
            
            domesticNPV = swapPoint.domesticLeg.NPV(discountCurve);
            foreignNPV = swapPoint.foreignLeg.NPV(foreignDCurve);
            
            temp_ccyRate= swapPoint.domesticAmount;
            
            t1 = DateDiff(swapPoint.maturityDate,discountCurve.params('asOfDate'))/365.0;
            
%             denom = swapPoint.foreignAmount*discountCurve.DF(t1);
            denom = 1.0*discountCurve.DF(t1);
            
            if strcmp(swapPoint.type,'Long')
                out.NPV = -1.0*domesticNPV + 1.0*temp_ccyRate*foreignNPV;
                out.fairSwapPoint =  out.NPV/denom;
            else
                out.NPV = -1.0*(-1.0*domesticNPV + 1.0*temp_ccyRate*foreignNPV);
                out.fairSwapPoint =  -1.0*out.NPV/denom;
            end
            
        end
    end
    
end



In [ ]:
function [totalvariance, impliedvolatility] = svi_surface(k, theta, rho, phifun, phi_param, tau)
%svi_surface calcualtes the surface SVI free of statis arbitrage.
%
% Input:
% * k = (Kx1) = moneyness at which to evaluate the surface
% * theta = (Tx1) = ATM variance time at which to evaluate the surface.
% If theta and k have the same dimensions, then the surface is evaluated 
% for each given (k,theta) pair. If the dimensions are different, the 
% function evaluates the surface on the grid given by k and theta
% * param = (Xx1) = parameters of the surface
%
% Output:
% * totalvariance = (KxT) or (Kx1) = estimated total variance
% * impliedvolatility = (KxT) or (Kx1) = estimated implied volatility

% ensure correct input
k = k(:);
theta = theta(:);
assert(numel(rho)==1, 'rho has to be a scalar');

if nargout==2 && nargin == 5
    error('Implied volatility calculation requires tau');
end

%evaluate phi
switch phifun
    case 'heston_like'
        phi = heston_like(theta, phi_param);
    case 'power_law'
        phi = power_law(theta, phi_param);
    otherwise
        error('Incorrect function for phi');
end
        

if isequal(size(k), size(theta))
    totalvariance = theta/2 .* (1 + rho * phi.*k + ...
        sqrt((phi.*k + rho).^2 + (1-rho^2)));
else
    T = length(theta);
    K = length(k);
    totalvariance = zeros(K,T);
    for t = 1:T
        totalvariance(:,t) = theta(t)/2 .* (1 + rho * phi(t).*k + ...
        sqrt((phi(t).*k + rho).^2 + (1-rho^2)));
    end
end

if nargout == 2
    if isequal(size(k), size(theta), size(tau))
        impliedvolatility = sqrt(totalvariance./tau);
    else
        % expand tau
        tau_expanded = zeros(size(totalvariance));
        maturities = unique(tau);
        T = length(maturities);
        for t = 1:T
            tau_expanded(:,t) = maturities(t);
        end
        impliedvolatility = sqrt(totalvariance./tau_expanded);
    end
end

end

function value = heston_like(theta, param)
% Heston-like parameterization
lambda = param(1);
value = 1./(lambda*theta) .* (1 - (1 - exp(-lambda*theta))./(lambda*theta));
end

function value = power_law(theta, param)
% Power-law parameterization
eta = param(1);
gamma = param(2);

value = eta./(theta.^(gamma) .* (1+theta).^(1-gamma));
end


In [ ]:
function [totalvariance, impliedvolatility] = svi_raw(k, param, tau)
%SVI - Stochastic Volatility Inspired parameterization of the implied
%volatility smile. This function implements the raw formulation.
% Input:
% * k = (Nx1) = log-moneyness at which to evaluate the total implied
% variance
% * param = (5x1) = parameters:
%   * a = scalar = level of variance
%   * b = scalar = slope of wings
%   * m = scalar = translates smile to right
%   * rho = scalar = counter-clockwise rotation of smile
%   * sigma = scalar = reduces ATM curvature of the smile
% Output:
% * totalvariance = (Nx1) = estimated total variance at k
% * impliedvolatility = (Nx1) = estimated implied volatility at (k,tau)

% make sure that input is in column vector format
k = k(:);

% check that input is consistent
assert(numel(param)==5, 'There have to be five parameters: a, b, m, rho, sigma');

if nargout==2 && nargin==2
    error('tau needs to be specified in order to return implied volatilities');
end

a = param(1);
b = param(2);
m = param(3);
rho = param(4);
sigma = param(5);
        
% make sure that parameter restrictions are satisfied
% assert(b >= 0, 'b has to be non-negative');
% assert(abs(rho) <= 1, '|rho| has to be smaller than 1');
% assert(sigma >= 0, 'sigma has to be positive');
% assert(a + b*sigma*sqrt(1-rho^2)>=0, 'a + b sigma (1-rho^2)^0.5 has to be non-negative');

% calculate total variance
totalvariance = a + b*(rho*(k-m) + sqrt((k-m).^2 + sigma^2));

% if requested, calculate implied volatility
if nargout == 2
    impliedvolatility = sqrt(totalvariance/tau);
end
    
end



In [ ]:
function [lb, ub] = svi_parameter_bounds(parameterization)
% svi_parameter_bounds returns the parameter bounds of an SVI parameterization. The parameters are
% assumed to be in the following order:
% raw = [a b m rho sigma]
% natural = [delta mu rho omega zeta]
% jumpwing = [v psi p c vt]

large = 1e5;
switch parameterization
    case 'raw'
        lb = [-large 0     -large -1 0    ];
        ub = [large  large  large  1 large];
    case 'natural'
        lb = [-large -large -1 0     0    ];
        ub = [ large  large  1 large large];
    case 'jumpwing'
        lb = [0     -large 0     0     0    ];
        ub = [large  large large large large];
    otherwise
        error('Unknown parameterization');
end

end

In [ ]:
function [totalvariance, impliedvolatility] = svi_natural(k, param, tau)
%SVI - Stochastic Volatility Inspired parameterization of the implied
%volatility smile. This function implements the natural formulation.
% Input:
% * k = (Nx1) = log-moneyness at which to evaluate the total implied
% variance
% * param = (5x1) = parameters:
%   * delta = scalar = level of variance
%   * mu = scalar = slope of wings
%   * rho = scalar = translates smile to right,
%   * omega = scalar = counter-clockwise rotation of smile
%   * zeta = scalar = reduces ATM curvature of the smile
% Output:
% * totalvariance = (Nx1) = estimated total variance at k
% * impliedvolatility = (Nx1) = estimated implied volatility at (k,tau)

% make sure that input is in column vector format
k = k(:);

% check that input is consistent
assert(numel(param)==5, 'There have to be five parameters: a, b, m, rho, sigma');

if nargout==2 && nargin==2
    error('tau needs to be specified in order to return implied volatilities');
end

delta = param(1);
mu = param(2);
rho = param(3);
omega = param(4);
zeta = param(5);

% make sure that parameter restrictions are satisfied
assert(omega >= 0, 'omega has to be non-negative');
assert(abs(rho) < 1, '|rho| has to be smaller than 1');
assert(zeta > 0, 'zeta has to be positive');
assert(delta + omega*(1-rho^2) >= 0, 'delta + omega (1-rho^2) has to be non-negative');

% calculate total variance
totalvariance = delta + omega/2*(1+ zeta*rho*(k-mu) + sqrt((zeta*(k-mu)+rho).^2+(1-rho^2)));

% if requested, calculate implied volatility
if nargout == 2
    impliedvolatility = sqrt(totalvariance/tau);
end
    
end



In [ ]:
function [totalvariance, impliedvolatility] = svi_jumpwing(k, param, tau)
%SVI - Stochastic Volatility Inspired parameterization of the implied
%volatility smile. This function implements the jump-wings formulation.
% Input:
% * k = (Nx1) = log-moneyness at which to evaluate the total implied
% variance
% * param = (5x1) = parameters:
%   * v = scalar = ATM variance
%   * psi = scalar = ATM skew
%   * p = scalar = slope of left/put wing
%   * c = scalar = slope of right/call wing
%   * vt = scalar = minimum implied variance
% Output:
% * totalvariance = (Nx1) = estimated total variance at k
% * impliedvolatility = (Nx1) = estimated implied volatility at (k,tau)

% make sure that input is in column vector format
k = k(:);

% check that input is consistent
assert(numel(param)==5, 'There have to be five parameters: v, psi, p, c, vt');

if nargin==2
    error('tau needs to be specified for jump-wings formulation');
end

v = param(1);
p = param(3);
c = param(4);
vt = param(5);
        
% make sure that parameter restrictions are satisfied
% assert(v >= 0, 'v has to be non-negative');
% assert(p >= 0, 'p has to be non-negative');
% assert(c >= 0, 'c has to be non-negative');
% assert(vt >= 0, 'vt has to be non-negative');

% convert parameters to raw formulation
[ param_raw ] = svi_convertparameters( param, 'jumpwing', 'raw', tau);

% calculate total variance
[totalvariance, impliedvolatility] = svi_raw(k, param_raw, tau);
    
end



In [ ]:
function [call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
    forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interest_rate_theta)
%svi_interpolation estimates inter/extrapolated SVI
%
%Input:
% * log_moneyness = (Kx1) = log-moneyness at which to evalute volatility slices
% * tau_interp = scalar = maturity at which to generate volatility slices
% * forward_interp = scalar = forward prices corresponding to maturities
% * interest_interp = scalar = interest rates corresponding to maturities
% * parameters (5xL) = estimated parameters of SVI in jumpwing parameterization
% * theta (Lx1) = ATM total variance time at which the parameters were estimated
% * maturities (Lx1) = time to maturity corresponding to theta
% * forward_theta = (Lx1) = forward prices corresponding to theta
% * interest_rate_theta = (Lx1) = interest rates corresponding to theta (can be scalar)
%
%Output:
% * total_implied_variance (Kx1) = total_implied_variances for each log_moneyness and tau_interp
% * implied_volatility (Kx1) = implied volatilities corresponding to total_implied_variance
% * call_price (Kx1) = call option prices correspongin to total_implied_variance

%ensure column vectors
log_moneyness = log_moneyness(:);
theta = theta(:);
maturities = maturities(:);
forward_theta = forward_theta(:);
interest_rate_theta = interest_rate_theta(:);

%ensure scalar input
assert(numel(tau_interp)==1, 'tau_interp has to be scalar');
assert(numel(forward_interp)==1, 'forward_interp has to be scalar');
assert(numel(interest_interp)==1, 'interest_interp has to be scalar');

%expand scalar input
if numel(interest_rate_theta)==1
    interest_rate_theta = interest_rate_theta*ones(size(theta));
end

% ensure correct size of input
assert(size(parameters,2)==size(theta,1), 'parameter set for each theta required');
assert(isequal(size(theta), size(forward_theta), size(interest_rate_theta)), ...
    'theta, forward_theta, and interestrate_theta have to have the same size');






% estimate theta for interpolated maturity
theta_interp = interp1(maturities, theta, tau_interp, 'linear', 'extrap');
close = forward_interp.*exp(-interest_interp*tau_interp);
if ismember(tau_interp, maturities)
    total_implied_variance = svi_jumpwing(k, parameters(:,maturities==tau_interp), tau_interp);
    implied_volatility = sqrt(total_implied_variance/tau_interp);
    strike = forward_interp*exp(log_moneyness);
%     call_price = blsprice(close, strike, interest_interp, tau_interp, implied_volatility);
    call_price = zeros(length(strike),1);
    
    for idxhh=1:length(strike)
        call_price(idxhh) = exp(-interest_interp*tau_interp)*blackPrice(forward_interp, strike(idxhh),implied_volatility(idxhh), tau_interp, 1);
    end
    
else
    if min(maturities) < tau_interp && tau_interp < max(maturities)
        % interpolation
        [~, idx] = min(abs(tau_interp-maturities));
        % if closest maturity is smaller than tau_interp, make idx one unit larger --> idx is index of
        % smallest maturity larger than tau_interp
        if maturities(idx) < tau_interp
            idx = idx+1;
        end
        
        alpha_t = (sqrt(theta(idx)) - sqrt(theta_interp))/(sqrt(theta(idx)) - sqrt(theta(idx-1)));
        param_interp = alpha_t*parameters(:,idx-1) + (1-alpha_t)*parameters(:,idx);
        total_implied_variance = svi_jumpwing(log_moneyness, param_interp, tau_interp);        
        implied_volatility = sqrt(total_implied_variance/tau_interp);
        strike = forward_interp*exp(log_moneyness);
%         call_price = blsprice(close, strike, interest_interp, tau_interp,implied_volatility);
%         call_price = exp(-interest_interp*tau_interp)*blackPrice(forward_interp, strike,implied_volatility, tau_interp, 1);
        call_price = zeros(length(strike),1);
    
        for idxhh=1:length(strike)
            call_price(idxhh) = exp(-interest_interp*tau_interp)*blackPrice(forward_interp, strike(idxhh),implied_volatility(idxhh), tau_interp, 1);
        end
        call_price(call_price<0) = 0;
        
        
    elseif tau_interp < min(maturities)
        % extrapolation for small maturities
        forward_0 = interp1(theta, forward_theta, 0, 'linear','extrap');
        strike_1 = forward_0*exp(log_moneyness);
        call_price_1 = max(close-strike_1,0);
        
        idx = 1;
        total_implied_variance_2 = svi_jumpwing(log_moneyness, parameters(:,idx), maturities(idx));
        implied_volatility_2 = sqrt(total_implied_variance_2/maturities(idx));
        strike_2 = forward_theta(idx)*exp(log_moneyness);
%         call_price_2 = blsprice(close, strike_2, interest_rate_theta(idx), maturities(idx), implied_volatility_2);
        call_price_2 = exp(-interest_rate_theta(idx)*maturities(idx))*blackPrice(forward_interp, strike_2,implied_volatility_2 , maturities(idx),1);
        
        call_price_2(call_price_2<0) = 0;
        
        alpha_t = (sqrt(theta(idx)) - sqrt(theta_interp))/sqrt(theta(idx));
        K_t = forward_interp * exp(log_moneyness);
        
        call_price = K_t .* (alpha_t*call_price_1./strike_1 + (1-alpha_t)*call_price_2./strike_2);
%         implied_volatility = blsimpv(close, K_t, interest_interp, tau_interp, call_price, []);
        implied_volatility = blackVolLBR(exp(interest_interp*tau_interp)*call_price,forward_interp, K_t, interest_interp, tau_interp,1);
        
        total_implied_variance = implied_volatility.^2*tau_interp;
    else
        % extrapolation for large maturities
        total_implied_variance = svi_jumpwing(log_moneyness, parameters(:,end), maturities(end));
        total_implied_variance = total_implied_variance + theta_interp-theta(end);
        implied_volatility = sqrt(total_implied_variance/tau_interp);
        strike = forward_interp*exp(log_moneyness);
%         call_price = blsprice(close, strike, interest_interp, tau_interp, implied_volatility);
        call_price = exp(-interest_interp*tau_interp)*blackPrice(forward_interp, strike, implied_volatility, tau_interp, 1);
        
    end
        
end
    
    
end

In [ ]:
function [ param_new ] = svi_convertparameters( param_old, from, to, tau)
%svi_convertparameters converts the parameter set of one type of SVI
%formulation to another. The parameterizations are assumed to be:
% * raw =(a,b,m,rho, sigma)
% * natural = (delta, mu, rho, omega, zeta)
% * jumpwing = (v, psi, p, c, vt)
%
% Input:
% * param_old = (5x1) = original parameters
% * from = string = formulation of original parameters (raw, natural,
% jumpwing)
% * to = string = formulation of new parameters (raw, natural, jumpwings)
%
% Output:
% param_new = (5x1) = new parameters

% test that input is correct
assert(numel(param_old)==5, 'There have to be five original parameters');

if ~(strcmp(from, 'raw') || strcmp(from, 'natural') || strcmp(from, 'jumpwing'))
    error('from has to be one of: raw, natural, jumpwing');
end

if ~(strcmp(to, 'raw') || strcmp(to, 'natural') || strcmp(to, 'jumpwing'))
    error('from has to be one of: raw, natural, jumpwing');
end

if (strcmp(to, 'jumpwing') || strcmp(from, 'jumpwing')) && nargin==3
    error('tau is required for tailwings formulation');
end

switch from
    case 'raw'
        a = param_old(1);
        b = param_old(2);
        m = param_old(3);
        rho = param_old(4);
        sigma = param_old(5);
        switch to
            case 'raw'
                param_new = param_old;
            case 'natural'
                omega = 2*b*sigma/sqrt(1-rho^2);
                delta = a - omega/2*(1-rho^2);
                mu = m + rho*sigma/sqrt(1-rho^2);
                zeta = sqrt(1-rho^2)/sigma;
                param_new = [delta; mu; rho; omega; zeta];
            case 'jumpwing'
                w = a + b*(-rho*m + sqrt(m^2+sigma^2));
                v = w/tau;
                psi = 1/sqrt(w) * b/2 * (-m/sqrt(m^2+sigma^2) + rho);
                p = 1/sqrt(w) * b * (1-rho);
                c = 1/sqrt(w) * b * (1+rho);
                vt = 1/tau * (a + b*sigma*sqrt(1-rho^2));
                param_new = [v; psi; p; c; vt];
        end
    case 'natural'
        
        switch to
            case 'raw'
                delta = param_old(1);
                mu = param_old(2);
                rho = param_old(3);
                omega = param_old(4);
                zeta = param_old(5);
                
                a = delta + omega/2*(1-rho^2);
                b = omega*zeta/2;
                m = mu - rho/zeta;
                sigma = sqrt(1-rho^2)/zeta;
                param_new = [a; b; m; rho; sigma];
            case 'natural'
                param_new = param_old;
            case 'jumpwing'
                param_temp = svi_convertparameters( param_old, 'natural', 'raw', tau);
                param_new = svi_convertparameters( param_temp, 'raw', 'jumpwing', tau);
        end
    case 'jumpwing'
        
        switch to
            case 'raw'
                v = param_old(1);
                psi = param_old(2);
                p = param_old(3);
                c = param_old(4);
                vt = param_old(5);
                w = v*tau;
                
                b = sqrt(w)/2 * (c+p);
                rho = 1-p*sqrt(w)/b;
                beta = rho - 2*psi*sqrt(w)/b;
                alpha = sign(beta)*sqrt(1/beta^2-1);
                m = (v-vt)*tau/(b*(-rho+sign(alpha)*sqrt(1+alpha^2)-alpha*sqrt(1-rho^2)));
                if m == 0
                    sigma = (vt*tau - w)/b / (sqrt(1-rho^2)-1);
                else
                    sigma = alpha*m;
                end
                a = vt*tau - b*sigma*sqrt(1-rho^2);
                
                if sigma < 0
                    sigma = 0;
                end

                param_new = [a; b; m; rho; sigma];
            case 'natural'
                param_temp = svi_convertparameters( param_old, 'jumpwing', 'raw', tau);
                param_new = svi_convertparameters( param_temp, 'raw', 'natural', tau);
            case 'jumpwing'
                param_new = param_old;
        end
end

end



In [ ]:
function [totalvariance, impliedvolatility] = svi(parameterization, k, param, tau)
%SVI - Stochastic Volatility Inspired parameterization of the implied
%volatility smile.
% Input:
% * parameterization = string = one of: raw, natural, or jumpwing
% * k = (Nx1) = log-moneyness at which to evaluate the total implied
% variance
% * param = (5x1) = parameters:
%   * a = scalar = level of variance
%   * b = scalar = slope of wings
%   * m = scalar = translates smile to right
%   * rho = scalar = counter-clockwise rotation of smile
%   * sigma = scalar = reduces ATM curvature of the smile
% Output:
% * totalvariance = (Nx1) = estimated total variance at k
% * impliedvolatility = (Nx1) = estimated implied volatility at (k,tau)

if nargout==2
    switch parameterization
        case 'raw'
            [totalvariance, impliedvolatility] = svi_raw(k, param, tau);
        case 'natural'
            [totalvariance, impliedvolatility] = svi_natural(k, param, tau);
        case 'jumpwing'
            [totalvariance, impliedvolatility] = svi_jumpwing(k, param, tau);
        otherwise
            error('Unknown parameterization specified');
    end
else
    switch parameterization
        case 'raw'
            totalvariance = svi_raw(k, param);
        case 'natural'
            totalvariance = svi_natural(k, param);
        case 'jumpwing'
            totalvariance = svi_jumpwing(k, param, tau);
        otherwise
            error('Unknown parameterization specified');
    end
end

end

In [ ]:
classdef SurvProbCurve < MktData
    %UNTITLED4 Summary of this class goes here
    %   Detailed explanation goes here
    
    properties(SetAccess = public)
        convention;
        asOfDate;
    end
    
    methods
        % constructor
        function survProbCurve = SurvProbCurve(MktData)
            if nargin > 0
                survProbCurve.params = MktData.params;
                if ~isKey(MktData.params,'rawData')
                    survProbCurve.params('rawData') = [0    1.0];
                end
                
                if ~isKey(MktData.params,'asOfDate')
                    survProbCurve.asOfDate = H_Date('1900-01-01');
                else
                    survProbCurve.asOfDate = MktData.params('asOfDate');
                end
                
                if ~isKey(MktData.params,'convention')
                    survProbCurve.convention = {};
                else
                    survProbCurve.convention = MktData.params('convention');
                end
                
            else
                survProbCurve.params = containers.Map();
                %default zero Curve
                survProbCurve.params('rawData') = [0    1.0];
                survProbCurve.params('asOfDate') = H_Date('1900-01-01');
                survProbCurve.convention = {};
            end     
        end
        
        % SurvProb functor is a linear interpolator of SurvProbCurve
        function    survProb = SurvProb(survProbCurve, t)
            
            rawData = survProbCurve.params('rawData');
            abscisses = rawData(:,1);
            ordinates = rawData(:,2);
            ordinatesLog = log(ordinates);
            
            %flat forward hazard rate extrapolation at the ends
            if t < 0
                error('t should be positive!')
            elseif t <= abscisses(1)
                slope = ordinatesLog(1)/abscisses(1);
                logSurvProb = ordinatesLog(1) + slope*(t-abscisses(1)); 
            elseif t <= abscisses(end)
                logSurvProb = H_interpolation(abscisses, ordinatesLog,t,1);
            else
                slope = (ordinatesLog(end)-ordinatesLog(end-1))/(abscisses(end)-abscisses(end-1));
                logSurvProb = ordinatesLog(end) + slope*(t-abscisses(end));
            end
            %extrapolation ??
            
            survProb = exp(logSurvProb);
        end
        
        function    survProb = SurvProbFromDate(survProbCurve, h_date)
            
            t = DateDiff(h_date,survProbCurve.params('asOfDate'))/365.0;
            survProb = survProbCurve.SurvProb(t);
        end
        
        function    defaultProb = DefaultProb(survProbCurve, t1,t2)
            if nargin > 2
                if t1 > t2
                    error('t1 should be before t2 !')
                end

                % cumulative default probability at t1
                p1 = 1.0 - survProbCurve.SurvProb(t1);
                p2 = 1.0 - survProbCurve.SurvProb(t2);
                % defaultProb in [t1,t2] conditional that

                defaultProb = p2 - p1;
            
            else
                % cumulative default probability at t1
                p1 = 1.0 - survProbCurve.SurvProb(t1);
                % defaultProb in [t1,t2] conditional that

                defaultProb = p1;
                
            end
            
        end
        
        function    defaultProb = DefaultProbFromDate(survProbCurve, h_date1,h_date2)
            if nargin > 2
                t1 = DateDiff(h_date1,survProbCurve.params('asOfDate'))/365.0;
                t2 = DateDiff(h_date2,survProbCurve.params('asOfDate'))/365.0;
                defaultProb = survProbCurve.DefaultProb(t1,t2);
            else
                t1 = DateDiff(h_date1,survProbCurve.params('asOfDate'))/365.0;
                defaultProb = survProbCurve.DefaultProb(t1);
            end
        end
            
    end
    
end



In [ ]:
classdef ZeroCurve < MktData
    %UNTITLED4 Summary of this class goes here
    %   Detailed explanation goes here
    
    properties(SetAccess = public)
        convention;
        asOfDate;
    end
    
    methods
        % constructor
        function zeroCurve = ZeroCurve(MktData)
            if nargin > 0
                zeroCurve.params = MktData.params;
                if ~isKey(MktData.params,'rawData')
                    zeroCurve.params('rawData') = [0    0.02];
                end
                
                if ~isKey(MktData.params,'asOfDate')
                    zeroCurve.asOfDate = H_Date('1900-01-01');
                else
                    zeroCurve.asOfDate = MktData.params('asOfDate');
                end
                
                if ~isKey(MktData.params,'convention')
                    zeroCurve.convention = {};
                else
                    zeroCurve.convention = MktData.params('convention');
                end
                
            else
                zeroCurve.params = containers.Map();
                %default zero Curve
                zeroCurve.params('rawData') = [0    0.02];
                zeroCurve.params('asOfDate') = H_Date('1900-01-01');
                zeroCurve.convention = {};
            end     
        end
        
        % zero functor is a linear interpolator of zeroCurve
        function    zero = Zero(zeroCurve, t)
            
            rawData = zeroCurve.params('rawData');
            abscisses = rawData(:,1);
            ordinates = rawData(:,2);
            zero = H_interpolation(abscisses, ordinates,t,1);
        end
        
        function    zero = ZeroFromDate(zeroCurve, h_date)
            
            t = DateDiff(h_date,zeroCurve.params('asOfDate'))/365.0;
            zero = zeroCurve.Zero(t);
        end
         
        function    df = DF(zeroCurve, t)
            df = exp(-zeroCurve.Zero(t)*t);
        end
        
            
    end
    
end



In [ ]:
classdef SpreadRangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenorL
        tenorS
        lower
        upper
        rangeDays
        callYN
        schedule
        couponSchedule
        callSchedule
        manualScheduleYN
        isExercised
    end
    
    methods
        % constructor
        function rangeAccrual = SpreadRangeAccrual(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                %rangeAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.rangeDays = params.params('rangeDays');
                rangeAccrual.callYN = params.params('callYN');
                rangeAccrual.manualScheduleYN = params.params('manualScheduleYN');
                if rangeAccrual.manualScheduleYN
                    rangeAccrual.couponSchedule = rangeAccrual.ManualScheduleGenerate(params.params('couponScheduleInt'));
                    rangeAccrual.callSchedule = rangeAccrual.ManualScheduleGenerate(params.params('callScheduleInt'));
                    
                
                else
                    maturityDate = params.startDate.AddDate(params.maturity,'year');
                    rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,maturityDate,rangeAccrual.couponFreq);
                
                end
                
                if isfield(params, 'isExercised')
                    rangeAccrual.isExercised = params.isExercised;
                else
                    rangeAccrual.isExercised = false;  % default value is false
                end
        end
        
        function out = ManualScheduleGenerate(rangeAccrual,scheduleInt)
            scheduleSize = size(scheduleInt,1);
            out.numOfPeriod = scheduleSize;
            for i=1:scheduleSize
                out.resetDates(i) = H_Date(scheduleInt(i,1));
                out.startDates(i) = H_Date(scheduleInt(i,2));
                out.endDates(i) = H_Date(scheduleInt(i,3));
                out.payDates(i) = H_Date(scheduleInt(i,4));
                out.dayCount(i) = out.endDates(i).DateDiff(out.startDates(i));
                out.dayCountFraction(i) = out.dayCount(i)/ 365.0;
            end
            
            
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCount = schedule.dayCount;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = numMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endM1TimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endM1TimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endM1TimeIdx(i-1) + 1;
                    out.endM1TimeIdx(i) = out.endM1TimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(rangeAccrual,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                sqrtDt = sqrt((nextStep-currentStep)/365.0);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 1;
            outInfo = rangeAccrual.numMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            numMethodInfo.timeStepsInfo = outInfo;
            
            % mc random number init
            %pathSize = 2^16;
            pathSize = 5000;
            
            %pathSize = 2^10;
            
            rng('default');
            rng(0);
            
            numOfFactors = 2;
            
            Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
            Z=[Z,-Z];
            
            numMethodInfo.pathSize = pathSize;
            numMethodInfo.numOfFactors = numOfFactors;
            numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
            numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
            numMethodInfo.Z = Z;
            
            % numeraire info
            numMatTime = numMethodInfo.totalTimeSteps(end);
            dfNumMat = model.DF(numMatTime/365.0);
            
            %generate model states for the given timeSteps
            modelStates = rangeAccrual.generateModelStates(valueDate,schedule,numMethodInfo,model);
            
            % induct backward and calculate cashflow
            
            
            periodCount = schedule.numOfPeriod;
            
            nonCall.npv = 0;
            
            nonCall.cashflow_e = zeros(periodCount + 1,1);
            nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
            nonCall.rangeProb_e = zeros(periodCount + 1,1);
            
            nonCall.df_e = zeros(periodCount + 1,1);
            nonCall.df_c = zeros(periodCount + 1,1);
            
            % sum of cashflow_npv
            nonCall.payoff = zeros(1,pathSize);
            
            nonCall.payoffStateVectors_cashflow = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVectors_cashflow_npv = zeros(periodCount+1,pathSize);
            nonCall.payoffStateVector_df = zeros(periodCount + 1,pathSize);
            
            payoffStateVectors_cashflow = zeros(1, pathSize);
            payoffStateVectors_cashflow_npv = zeros(1, pathSize);
            payoffStateVectors_df = zeros(1,pathSize);
            
            % we calculate cashflow backward from maturityDate to valueDate
            % rangeAccrual coupons are added at the startDate of the Period
            % At maturity
            
            nominal = rangeAccrual.nominal;
            coupon = rangeAccrual.coupon;
            
            currentTime = numMethodInfo.totalTimeSteps(end);
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            currentTimeIdx = totalTimeStepsSize;
            
            modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
            payoffStateVectors_cashflow = nominal*ones(1,pathSize);
            nonCall.payoffStateVectors_cashflow(end,:) = payoffStateVectors_cashflow;
            
            payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
            nonCall.payoffStateVectors_cashflow_npv(end,:) = payoffStateVectors_cashflow_npv;
            
            nonCall.cashflow_npv_e(end) = mean(payoffStateVectors_cashflow_npv);
            
            nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
            
            payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
            
            nonCall.payoffStateVector_df(end,:) = payoffStateVectors_df;
            nonCall.df_e(end) = mean(nonCall.payoffStateVector_df(end,:));
            nonCall.df_c(end) = model.DF(currentTime/365.0);
%             
%             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
%             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));
            
            lastTimeIdx = currentTimeIdx;
            totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
            
            for i=periodCount:-1:1
                % past schedule neglect
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                endM1TimeIdx = numMethodInfo.timeStepsInfo.endM1TimeIdx(i);
                
                %% FromToInduction start
                
                %modelsStates induct backward(lastTimeIdx to endM1TimeIdx)
                while endM1TimeIdx < lastTimeIdx
                    lastTimeIdx = lastTimeIdx -1;
                    if lastTimeIdx == endM1TimeIdx break;end
                end
                
                currentTimeIdx = lastTimeIdx;
                currentTime = totalTimeSteps(currentTimeIdx);
                % now we are at endM1TimeIdx
                % variables for coupon valuation initialized
                ralower = rangeAccrual.lower;
                raupper = rangeAccrual.upper;
                tenorLong = rangeAccrual.tenorL;
                tenorShort = rangeAccrual.tenorS;
                
                cD = 0; % cumulated day count fraction
                cP = 0; 
                dI = 0; % dummy idx for loop
                dcfC = 0;
                dcfN = 0;
                dD = 0;
                idxL = zeros(1,pathSize);
                idxS = zeros(1,pathSize);
                spreadC = zeros(1,pathSize);
                spreadN = zeros(1,pathSize);
                probSpread = zeros(1,pathSize);
                simpleYN = true;
                numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                
                for j=numOfTimeSteps:-1:1
                    % evaluate dealdescription
                    % last step is omitted
                    currentTime = totalTimeSteps(currentTimeIdx);
                    dcfC = currentTime/365.0;
                    if j~= numOfTimeSteps
                        dD = (dcfN-dcfC);
                    end
                    
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    spreadC = idxL - idxS;
                     
                    probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                    cD = cD + dD;
                    cP = cP + probSpread;
  
                    dcfN = dcfC;
                    spreadN = spreadC;
                    
                    %induct backward to the next time step
                    % one time backward induction
                    lastTimeIdx = currentTimeIdx;
                    
                    if startTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        currentTimeIdx = lastTimeIdx;
                    end
                    
                    
                end
                
                %% FromToInduction end
                %% AtDate operation
                % Adding a new cashflow to the payoff
                
                dcf = schedule.dayCountFraction(i);
                payoffStateVectors_cashflow = nominal * coupon * dcf * cP/cD;
                
                % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                % i.e. currentTimeIdx has not changed
                % no need to upodate modelStateT
                % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                % since cashflow's are added at the startDate(i) we need to
                % multiply cashflow with the df(startDate(i), payDate(i))
                payTime = DateDiff(schedule.payDates(i),valueDate);
                df_se = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                
                for k=1:pathSize
                    payoffStateVectors_cashflow(k) = payoffStateVectors_cashflow(k) + df_se(k);
                end
                
                nonCall.payoffStateVectors_cashflow(i,:) = payoffStateVectors_cashflow;
                
                nonCall.cashflow_e(i) = mean(payoffStateVectors_cashflow);
                nonCall.rangeProb_e(i) = mean(nonCall.cashflow_e(i)/(nominal * coupon * dcf));
                
                payoffStateVectors_cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,payoffStateVectors_cashflow);
            
                nonCall.cashflow_npv_e(i) = mean(payoffStateVectors_cashflow_npv);
                
                nonCall.payoffStateVectors_cashflow_npv(i,:) = payoffStateVectors_cashflow_npv;
                nonCall.payoff = nonCall.payoff + payoffStateVectors_cashflow_npv;
                
                payoffStateVectors_df = model.discountPayoff(currentTime,numMatTime ...
                ,modelStateT,1.0*ones(1,pathSize));
                
                nonCall.payoffStateVector_df(i,:) = payoffStateVectors_df;
                nonCall.df_e(i) = mean(nonCall.payoffStateVector_df(i,:));
                nonCall.df_c(i) = model.DF(currentTime/365.0);
                
            end
            
            nonCall.npv = mean(nonCall.payoff);
  
            out.cashflow= nonCall.cashflow_e;
            out.cashflowNpv = nonCall.cashflow_npv_e;
            out.df_e = nonCall.df_e;
            out.df_c = nonCall.df_c;
            out.daycountFraction = schedule.dayCountFraction(i);
            out.rangeProb = nonCall.rangeProb_e;
            out.npv = nonCall.npv;
           
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = rangeAccrual.rangeDays;
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    toDate = observeDate;
                    tpDate = schedule.payDates(i);
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    rangeValue = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper);
%                     rangeValue = 1.0;
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                % bug fixed observeDays -> days
                rangeProb(i) = rangeProb(i)/days;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
%         function out = discountPayoff(rangeAccrual,eventTime,numMatTime ...
%                 ,model,modelStatesT,cashflow)
%             
%             dfNumMat = model.DF(numMatTime/365.0);
%             % for discountPayoff T=numMatTime
%             % P(0,TnumMat)/P(Te,Tnummat)*Payoff
%             %
%             discountT = model.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
%             stateSize = size(modelStatesT,2);
%             payoff = zeros(1,stateSize);
%             for i=1:stateSize
%                 payoff(i)= dfNumMat/discountT(i)*cashflow(i);
%             end    
%             out = payoff;
%         end
        
        
        
    end
    
end



In [ ]:
classdef SpreadCDSHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        spreadCDSSettlementDays
        fixingCalendar
        spreadCDSBasis
        spreadCDSConvention
        spreadCDSPeriod
        spreadCDSDateGenerationRule
        recoveryRate
        discountCurve
        
        evaluationDate
        
        settlesAccrual
        paysAtDefaultTime
        
        engineType
        
        creditDefaultSwap
        schedule
        survProbCurve
%         discountCurve
        protectionStartDate
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function spreadCDSHelper = SpreadCDSHelper(params)
                if nargin > 0
                    spreadCDSHelper.rateType = 'SpreadCDS';
                    
                    spreadCDSHelper.rate = params('rate');
                    spreadCDSHelper.rateTenor = params('rateTenor');
                    
                    spreadCDSHelper.spreadCDSSettlementDays = params('spreadCDSSettlementDays');
                    spreadCDSHelper.fixingCalendar = params('fixingCalendar');
                    spreadCDSHelper.spreadCDSBasis = params('spreadCDSBasis');
                    spreadCDSHelper.spreadCDSConvention = params('spreadCDSConvention');
                    spreadCDSHelper.spreadCDSPeriod = params('spreadCDSPeriod');
                    spreadCDSHelper.spreadCDSDateGenerationRule = params('spreadCDSDateGenerationRule');
                    spreadCDSHelper.recoveryRate  = params('recoveryRate');
                    spreadCDSHelper.discountCurve = params('discountCurve');
                    
                    spreadCDSHelper.evaluationDate = spreadCDSHelper.discountCurve.asOfDate;
                    
                    if ~isKey(params,'settlesAccrual')
                        spreadCDSHelper.settlesAccrual = true;
                    else
                        spreadCDSHelper.settlesAccrual = params('settlesAccrual');
                    end
                    
                    if ~isKey(params,'paysAtDefaultTime')
                        spreadCDSHelper.paysAtDefaultTime = true;
                    else
                        spreadCDSHelper.paysAtDefaultTime = params('paysAtDefaultTime');
                    end
                    
                    if ~isKey(params,'engineType')
                        spreadCDSHelper.engineType = 'Mid';
                    else
                        spreadCDSHelper.engineType = params('engineType');
                    end
                    
%                     spreadCDSHelper.discountCurve = params('discountCurve');
                    
%                     if spreadCDSHelper.discountCurve.asOfDate ~= iborIndex.forwardCurve.asOfDate
%                         error('forwardCurve asOfDate is different from discountCurve asOfDate')
%                     end
                    
                    spreadCDSHelper.creditDefaultSwap = MakeCreditDefaultSwap(spreadCDSHelper);
%                     spreadCDSHelper.schedule = MakeSchedule(spreadCDSHelper);
                    spreadCDSHelper.survProbCurve = SurvProbCurve();
                    spreadCDSHelper.survProbCurve.params('asOfDate') = spreadCDSHelper.discountCurve.asOfDate;
                else
                    spreadCDSHelper.rate = 0.02;
%                     spreadCDSHelper.iborIndex = IborIndex();
                end
                
                out = InitializeDates(spreadCDSHelper);
                
                spreadCDSHelper.earliestDate = out.earliestDate;
                spreadCDSHelper.latestDate = out.latestDate;
                
        end
        
        function out = MakeCreditDefaultSwap(spreadCDSHelper)
            referenceDate = spreadCDSHelper.evaluationDate;
            
            % for CDS start date is not adjusted
            spreadCDSHelper.protectionStartDate = referenceDate.AddDate(spreadCDSHelper.spreadCDSSettlementDays,'day');
%            startDate = spreadCDSHelper.fixingCalendar.Adjust(protectionStartDate,spreadCDSHelper.spreadCDSConvention);
            
            startDate = spreadCDSHelper.protectionStartDate;
            endDate = AddTenor(startDate,spreadCDSHelper.rateTenor);
            % fixed & floating schedule 
            fixedSchedule = Schedule(startDate,endDate,spreadCDSHelper.spreadCDSPeriod ,spreadCDSHelper.fixingCalendar,...
                                    spreadCDSHelper.spreadCDSConvention, spreadCDSHelper.spreadCDSDateGenerationRule,false); 
                                
            spreadCDSHelper.schedule = fixedSchedule;
             
            type = 'ProtectionBuyer';
            engineType = spreadCDSHelper.engineType;
            nominal = 100.0;
            spread = 0.01;
            
            creditDefaultSwap = CreditDefaultSwap(type,engineType,nominal,fixedSchedule,spread,spreadCDSHelper.spreadCDSBasis, ...
                                     spreadCDSHelper.spreadCDSConvention,spreadCDSHelper.settlesAccrual,...
                                     spreadCDSHelper.paysAtDefaultTime,spreadCDSHelper.protectionStartDate);
                                  
            out = creditDefaultSwap;
        end
        
        function out = MakeSchedule(spreadCDSHelper)
            referenceDate = spreadCDSHelper.evaluationDate;
            spreadCDSHelper.protectionStartDate = referenceDate.AddDate(spreadCDSHelper.spreadCDSSettlementDays,'day');
            
%             startDate = spreadCDSHelper.fixingCalendar.Adjust(protectionStartDate,spreadCDSHelper.spreadCDSConvention);
            % for CDS start date is not adjusted
            
            startDate = spreadCDSHelper.protectionStartDate;
            endDate = AddTenor(startDate,spreadCDSHelper.rateTenor);
            % fixed & floating schedule 
            schedule = Schedule(startDate,endDate,spreadCDSHelper.spreadCDSPeriod ,spreadCDSHelper.fixingCalendar,...
                                    spreadCDSHelper.spreadCDSConvention, spreadCDSHelper.spreadCDSDateGenerationRule,false); 
            
                                  
             out = schedule;
        end
        
        function out = InitializeDates(spreadCDSHelper)
            
            %earliestDate
            earliestDate = spreadCDSHelper.schedule.dates{1};
            latestDate = spreadCDSHelper.fixingCalendar.Adjust(spreadCDSHelper.schedule.dates{end},spreadCDSHelper.spreadCDSConvention);
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(spreadCDSHelper)
            outInfo = spreadCDSHelper.creditDefaultSwap.Calculate(spreadCDSHelper.recoveryRate,...
                                      spreadCDSHelper.survProbCurve,spreadCDSHelper.discountCurve);
%             out = outInfo.fairRate;
            out = outInfo.fairSpread;
            
        end
        
        % bootstrapping
        function out = QuoteError(spreadCDSHelper)
            out = spreadCDSHelper.rate - spreadCDSHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(spreadCDSHelper,guess,idx)
            spreadCDSHelper.Update(idx,guess);
            out = spreadCDSHelper.QuoteError();
        end
        
        function Update(spreadCDSHelper,idx,guess)
             % forwardCurve update
            forwardRawData = spreadCDSHelper.survProbCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            spreadCDSHelper.survProbCurve.params('rawData')= forwardRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef SouthKorea_Settlement < Calendar

    

    properties(SetAccess = public)
        
    end

    

    methods

        function southKorea = SouthKorea_Settlement(Calendar)
            southKorea.name = Calendar.name;
            southKorea.holidaysList = Calendar.holidaysList;
        end
  
        function out = IsHoliday(southKorea,h_Date)

            if strcmp(class(h_Date),'char') || strcmp(class(h_Date),'string')

                h_Date = H_Date(h_Date);

            end

            
            % fixed Date holiday
            if IsWeekend(h_Date) ...
               || h_Date.day == 1 && h_Date.month == 1 ...
               || h_Date.day == 1 && h_Date.month == 3 ...
               || h_Date.day == 5 && h_Date.month == 4 && h_Date.year <= 2005 ...
               || h_Date.day == 1 && h_Date.month == 5 ...
               || h_Date.day == 5 && h_Date.month == 5 ...
               || h_Date.day == 6 && h_Date.month == 6 ...
               || h_Date.day == 16 && h_Date.month == 7 && h_Date.year <= 2007 ...
               || h_Date.day == 15 && h_Date.month == 8 ...
               || h_Date.day == 3 && h_Date.month == 10 ...
               || h_Date.day == 25 && h_Date.month == 12
               
                out = true;
                return;
            % hangeul day and lunar new year    
            elseif h_Date.day == 9 && h_Date.month == 10 && h_Date.year > 2012 ...
                   || (h_Date.day == 27 || h_Date.day == 30) && h_Date.month == 1 && h_Date.year == 2017 ...
                   || (h_Date.day == 15 || h_Date.day == 16) && h_Date.month == 2 && h_Date.year == 2018 ...
                   || (h_Date.day == 4 || h_Date.day == 5 || h_Date.day == 6) && h_Date.month == 2 && h_Date.year == 2019 ...
                   || (h_Date.day == 24) && h_Date.month == 1 && h_Date.year == 2020 ...
                   || (h_Date.day == 11 || h_Date.day == 12) && h_Date.month == 2 && h_Date.year == 2021 ...
                   || (h_Date.day == 1 || h_Date.day == 2) && h_Date.month == 2 && h_Date.year == 2022 ...
                   || (h_Date.day == 23) && h_Date.month == 1 && h_Date.year == 2023 ...
                   || (h_Date.day == 9) && h_Date.month == 2 && h_Date.year == 2024 ...
                   || (h_Date.day == 28 || h_Date.day == 29 || h_Date.day == 30) && h_Date.month == 1 && h_Date.year == 2025
                
                out = true;
                return;
            % Budha's birthday
            elseif h_Date.day == 3 && h_Date.month == 5 && h_Date.year == 2017 ...
                   || h_Date.day == 22 && h_Date.month == 5 && h_Date.year == 2018 ...
                   || h_Date.day == 12 && h_Date.month == 5 && h_Date.year == 2019 ...
                   || h_Date.day == 30 && h_Date.month == 4 && h_Date.year == 2020 ...
                   || h_Date.day == 19 && h_Date.month == 5 && h_Date.year == 2021 ...
                   || h_Date.day == 15 && h_Date.month == 5 && h_Date.year == 2024 
                   
                out = true;
                return;
            % Election Days
            elseif h_Date.day == 9 && h_Date.month == 5 && h_Date.year == 2017 ...
                   || h_Date.day == 13 && h_Date.month == 6 && h_Date.year == 2018 ...
                   || h_Date.day == 15 && h_Date.month == 4 && h_Date.year == 2020 ...
                   || h_Date.day == 9 && h_Date.month == 3 && h_Date.year == 2022 
                   
                out = true;
                return;
            % Harvest Moon Day
            elseif (h_Date.day == 3 || h_Date.day == 4 || h_Date.day == 5) && h_Date.month == 10 && h_Date.year == 2017 ...
                   || (h_Date.day == 24 || h_Date.day == 25 || h_Date.day == 26) && h_Date.month == 9 && h_Date.year == 2018 ...
                   || (h_Date.day == 12 || h_Date.day == 13) && h_Date.month == 9 && h_Date.year == 2019 ...
                   || (h_Date.day == 30) && h_Date.month == 9 && h_Date.year == 2020 ...
                   || (h_Date.day == 1 || h_Date.day == 2) && h_Date.month == 10 && h_Date.year == 2020 ...
                   || (h_Date.day == 20 || h_Date.day == 21 || h_Date.day == 22) && h_Date.month == 9 && h_Date.year == 2021 ...
                   || (h_Date.day == 9 || h_Date.day == 12) && h_Date.month == 9 && h_Date.year == 2022 ...
                   || (h_Date.day == 28 || h_Date.day == 29) && h_Date.month == 9 && h_Date.year == 2023 ...
                   || (h_Date.day == 16 || h_Date.day == 17 || h_Date.day == 18) && h_Date.month == 9 && h_Date.year == 2024 ...
                   || (h_Date.day == 6 || h_Date.day == 7 || h_Date.day == 8) && h_Date.month == 10 && h_Date.year == 2025 
                   
                out = true;
                return;
            else

                index = cellfun(@(x) x==h_Date, southKorea.holidaysList, ...
                        'UniformOutput', 1);

                idx = find(index);

                if ~isempty(idx) 

                    out = true;

                    return;

                end

            end

            out = false;

        end


    end

    

end





In [ ]:
function [y,Dalpha_y,t] = SolveVIE(f,alpha,T,N,M)
% Description: Solves the Volterra integral equation (VIE)
%              
%               y(t) = (1/gamma(alpha))*int_0^t(t-u)^(alpha - 1)f(u,y(u))du
%
%               for y(t) and that assuming 0 < alpha < 1 with f an input 
%               function. The equation is solved for all t on the interval 
%               0 <= t <= T. It is the user's responsibility to ensure that 
%               the equation actually has a (unique) solution and does not 
%               blow up on the interval [0,T].
%
%               The equation is solved on the equidistant grid
%               0 < T/N < 2*T/N < 3*T/N < ... < N*T/N = T
%
%               The notation is the same as used in (Diethelm, 2004).
% 
% Parameters:
%   f:      [function] Function of two variables from the VIE. We allow it 
%           to return a [Mx1] vector but then it should also accept [Mx1]
%           vectors for its second argument.
%   alpha:  [1x1 real] Number between 0 and 1.
%   T:      [1x1 real] Upper time point to solve VIE on.
%   N:      [1x1 integer] Number steps to discretize [0,T] into.
%   M:      [1x1 integer, optional] Output dimension of f (for speed).
%
% Output: 
%   y:        [Mx(N+1) real] Solution of VIE, 
%             i.e. [y(0),y(T/N),y(2*T/N),...,y(T)]
%   Dalpha_y: [Mx(N+1) real] Fractional (alpha) derivative of solution, 
%             i.e. [D^(alpha)y(0),D^(alpha)y(T/N),...,D^(alpha)y(T)]
%   t:        [1x(N+1) real] Discretization, i.e. [0,T/N,2*T/N,...,T]
%
% References:
%   - Kai Diethelm, Neville J. Ford and Alan D. Freed, Detailed error
%   analysis for a fractional ADAMs method, Numerical Algorithms 36:31-52,
%   2004.
%
    
    if ~exist('M','var') || isempty(M)
        M = size(f(0,0),1);
    end
    
    % Initialization:
    h = T / N;
    t = (0:h:T);
    [y,Dalpha_y] = deal(NaN(M,N+1));
    
    % Define coefficient functions:
    dummy1 = ( (h^alpha) / (alpha*(alpha + 1)) );
    a_0_kp1 = @(k)( dummy1 *( k.^(alpha+1)-(k - alpha).*((k+1).^(alpha))));
    a_j_kp1 = @(j,k)( dummy1*( (k-j+2).^(alpha+1) ...
                      + (k-j).^(alpha+1) - 2*(k-j+1).^(alpha+1) ) );
    a_kp1_kp1 = dummy1;
    b_j_kp1 = @(j,k) (  ((h^alpha)/alpha) * ( (k+1-j).^(alpha) ...
                                            - (k-j).^(alpha)  ) );
    
    % Run scheme:
    y(:,1) = 0;
    Dalpha_y(:,1) = f(0,0);
    for k=0:N-1
        js = (0:1:k); 
        
        % Compute predictor:
        yp = sum(b_j_kp1(js,k).*Dalpha_y(:,1:k+1),2)./gamma(alpha);
        
        % Compute solution:
        if k==0
            y(:,2) = (a_0_kp1(k)*Dalpha_y(:,1) ...
                      + a_kp1_kp1*f(t(k+2),yp))./gamma(alpha);
        else
            y(:,k+2) = (a_0_kp1(k)*Dalpha_y(:,1) ...
                        + sum(Dalpha_y(:,2:k+1).*a_j_kp1(js(2:end),k),2)...
                        + a_kp1_kp1*f(t(k+2),yp))./gamma(alpha);
        end
        
        % Compute fractional derivative:
        Dalpha_y(:,k+2) = f(h,y(:,k+2));
        
    end
    
    if any(any(isnan(y))) || any(any(isnan(Dalpha_y)))
        error('SolveVIE: NaNs produced!');
    end
    
end



In [ ]:
classdef SingleCurrencyBasisSwapHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        gearing
        spread

        %Libor6M leg
        floatingLegTenor_L
        floatingLegConvention_L
        floatingLegDayCounter_L
        spotLag_L
        payLag_L

        % Libor3M Leg
        floatingLegTenor_S
        floatingLegConvention_S
        floatingLegDayCounter_S
        spotLag_S
        payLag_S
        
        liborIndex_L
        liborIndex_S
        
        evaluationDate
        
        vanilla_Floating_Floating_BasisSwap

        discountCurve
        
        calcCalendar
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function singleCurrencyBasisSwapHelper = SingleCurrencyBasisSwapHelper(params)
                if nargin > 0
                    singleCurrencyBasisSwapHelper.rateType = 'SingleCurrencyBasisSwap';
                    
                    singleCurrencyBasisSwapHelper.rate = params('rate');
                    singleCurrencyBasisSwapHelper.rateTenor = params('rateTenor');
                   
                    singleCurrencyBasisSwapHelper.floatingLegTenor_L = params('floatingLegTenor_L');
                    singleCurrencyBasisSwapHelper.floatingLegConvention_L = params('floatingLegConvention_L');
                    singleCurrencyBasisSwapHelper.floatingLegDayCounter_L = params('floatingLegDayCounter_L');
                    singleCurrencyBasisSwapHelper.spotLag_L = params('spotLag_L');
                    singleCurrencyBasisSwapHelper.payLag_L  = params('payLag_L');
                    
                    singleCurrencyBasisSwapHelper.floatingLegTenor_S = params('floatingLegTenor_S');
                    singleCurrencyBasisSwapHelper.floatingLegConvention_S = params('floatingLegConvention_S');
                    singleCurrencyBasisSwapHelper.floatingLegDayCounter_S = params('floatingLegDayCounter_S');
                    singleCurrencyBasisSwapHelper.spotLag_S = params('spotLag_S');
                    singleCurrencyBasisSwapHelper.payLag_S  = params('payLag_S');
                    
                    singleCurrencyBasisSwapHelper.liborIndex_L = params('liborIndex_L');
                    singleCurrencyBasisSwapHelper.liborIndex_S = params('liborIndex_S');
                    
                    singleCurrencyBasisSwapHelper.evaluationDate = singleCurrencyBasisSwapHelper.liborIndex_L.forwardCurve.asOfDate;
                    
%                     crossCurrencySwapHelper.domesticDCurve = params('domesticDCurve');
                    singleCurrencyBasisSwapHelper.discountCurve = params('discountCurve');
                    singleCurrencyBasisSwapHelper.calcCalendar = params('calcCalendar');
                    
                    
                    % can be generalized later for dual curve input
                    
%                     crossCurrencySwapHelper.foreignDCurve = crossCurrencySwapHelper.foreignIndex.forwardCurve;
%                     singleCurrencyBasisSwapHelper.foreignDCurve = params('foreignDCurve');
                    
                    if singleCurrencyBasisSwapHelper.discountCurve.asOfDate ~= singleCurrencyBasisSwapHelper.liborIndex_L.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    singleCurrencyBasisSwapHelper.vanilla_Floating_Floating_BasisSwap = MakeSingleCurrencyBasisSwap(singleCurrencyBasisSwapHelper);
                
                    out = InitializeDates(singleCurrencyBasisSwapHelper);

                    singleCurrencyBasisSwapHelper.earliestDate = out.earliestDate;
                    singleCurrencyBasisSwapHelper.latestDate = out.latestDate;
                end
        end
        
        function out = MakeSingleCurrencyBasisSwap(singleCurrencyBasisSwapHelper)
            
            liborIndex_L = singleCurrencyBasisSwapHelper.liborIndex_L;
            liborIndex_S = singleCurrencyBasisSwapHelper.liborIndex_S;
            referenceDate = liborIndex_L.forwardCurve.asOfDate;
            calcCalendar = singleCurrencyBasisSwapHelper.calcCalendar;
            
            startDate = calcCalendar.Advance(referenceDate,liborIndex_L.convention,...
                                                singleCurrencyBasisSwapHelper.spotLag_L,'day',false);
            
            endDate = AddTenor(startDate,singleCurrencyBasisSwapHelper.rateTenor);
            
            useEndOfMonth = false;
            if IsEndOfMonth(startDate)
                useEndOfMonth = true;
            end

            % fixed & floating schedule
            floatingLegSchedule_L = ScheduleWTermConvention(startDate,endDate,singleCurrencyBasisSwapHelper.floatingLegTenor_L,calcCalendar,...
                                        singleCurrencyBasisSwapHelper.floatingLegConvention_L,singleCurrencyBasisSwapHelper.floatingLegConvention_L,...
                                        'Backward',useEndOfMonth);

            floatingLegSchedule_S = ScheduleWTermConvention(startDate,endDate,singleCurrencyBasisSwapHelper.floatingLegTenor_S,calcCalendar,...
                                        singleCurrencyBasisSwapHelper.floatingLegConvention_S,singleCurrencyBasisSwapHelper.floatingLegConvention_S,...
                                        'Backward',useEndOfMonth);

             type = 'Payer';
             nominal = 1.0;
             
             % BasisSwap Rate
             floatingLegSpread_L = 0.0;
             floatingLegSpread_S = 0.0;
%              floatingLegSpread_S = singleCurrencyBasisSwapHelper.rate;

                

             vanilla_Floating_Floating_BasisSwap = Vanilla_Floating_Floating_BasisSwap(type,nominal,...
                                      floatingLegSchedule_L,liborIndex_L,...
                                      floatingLegSpread_L,singleCurrencyBasisSwapHelper.floatingLegDayCounter_L,...
                                      singleCurrencyBasisSwapHelper.spotLag_L,singleCurrencyBasisSwapHelper.payLag_L, ...
                                      floatingLegSchedule_S,liborIndex_S,...
                                      floatingLegSpread_S,singleCurrencyBasisSwapHelper.floatingLegDayCounter_S,...
                                      singleCurrencyBasisSwapHelper.spotLag_S,singleCurrencyBasisSwapHelper.payLag_S);
             out = vanilla_Floating_Floating_BasisSwap;
        end
        
        function out = InitializeDates(singleCurrencyBasisSwapHelper)
            
            %earliestDate
            earliestDate = singleCurrencyBasisSwapHelper.vanilla_Floating_Floating_BasisSwap.StartDate();
            latestDate = singleCurrencyBasisSwapHelper.vanilla_Floating_Floating_BasisSwap.EndDate();
            
            % need to be implemented ??
            % check later !!
            
%             floatEndValueDate = crossCurrencySwapHelper.vanillaSwap.floatingLeg.couponRates{end}.fixingEndDate; 
%             % if the last floating Index fixingEndDate is after
%             % latestDate(last accrualEndDate then we change
%             % the lastestDate into floatEndValueDate
%             
%             if DateDiff(floatEndValueDate,latestDate) > 0
%                 latestDate = floatEndValueDate;
%             end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(singleCurrencyBasisSwapHelper)
            outInfo = singleCurrencyBasisSwapHelper.vanilla_Floating_Floating_BasisSwap.Calculate(singleCurrencyBasisSwapHelper.discountCurve,...
                                                                                                singleCurrencyBasisSwapHelper.discountCurve);
            out = outInfo.fairSpread;
            
        end
        
        % bootstrapping
        function out = QuoteError(singleCurrencyBasisSwapHelper)
            out = singleCurrencyBasisSwapHelper.rate - singleCurrencyBasisSwapHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(singleCurrencyBasisSwapHelper,guess,idx)
            singleCurrencyBasisSwapHelper.Update(idx,guess);
            out = singleCurrencyBasisSwapHelper.QuoteError();
        end
        
        function Update(singleCurrencyBasisSwapHelper,idx,guess)
             % forwardCurve update
            forwardRawData = singleCurrencyBasisSwapHelper.liborIndex_L.forwardCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            singleCurrencyBasisSwapHelper.liborIndex_L.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
%             % discountCurve update
%             discountRawData = crossCurrencySwapHelper.discountCurve.params('rawData');
%             discountRawData(idx,2) = guess;
%             crossCurrencySwapHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
function [ S ] = SimulateOrnsteinUhlenbeckRough( S0, mu, sigma, lambda,deltat, t ) 
%% Approximate Ornstein-Uhlenbeck Generator. A more accurate version is preferred 
%% and available : SimulateOrnsteinUhlenbeck. 
%% License
% Copyright 2010, William Smith, CommodityModels.com . All rights reserved. 
% 
% Redistribution and use in source and binary forms, with or without modification, are 
% permitted provided that the following conditions are met: 
% 
% 1. Redistributions of source code must retain the above copyright notice, this list of 
% conditions and the following disclaimer. 
% 
% 2. Redistributions in binary form must reproduce the above copyright notice, this list 
% of conditions and the following disclaimer in the documentation and/or other materials 
% provided with the distribution. 
% 
% THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDER, WILLIAM SMITH ``AS IS'' AND ANY EXPRESS 
% OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF 
% MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL 
% THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, 
% SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT 
% OF SUBSTITUTE GOODS ORSERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) 
% HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, 
% OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS 
% SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. 


periods = floor(t / deltat); 

S = zeros(periods, 1); 

S(1) = S0; 

dWt = sqrt(deltat) * randn(periods,1); 

for t=2:1:periods 
    dSt = lambda*(mu-S(t-1))*deltat + sigma*dWt(t); 
    S(t) = S(t-1)+dSt; 
end

% OPTIM Note : 
% Precalculating all dWt's rather than one-per loop makes this function 

% approx 50% faster. Useful for Monte-Carlo simulations. 
% OPTIM Note : I tried calculating an array of dSt's and only doing a cumsum() at 
% the end, but it doesn't speedup any more. 

end


In [ ]:
function [ S ] = SimulateOrnsteinUhlenbeck( S0, mu, sigma, lambda, deltat, t ) 
%% Simulate an ornstein uhlenbeck process. 
%% Looks more complicated than expected, because if we don't include the 
%% exp() terms, we are not accurate as deltat becomes large. 

%% Reference 
% Based on the equation described in see 
% http://www.puc-rio.br/marco.ind/sim_stoc_proc.html#mc-mrd 
% For a formal treatment, see % Gillespie, D. T. 1996. 'Exact numerical simulation of the Ornstein-Uhlenbeck process 
% and its integral.' Physical review E 54, no. 2: 2084?2091. 

%% License % Copyright 2010, William Smith, CommodityModels.com . All rights reserved. 
% % Redistribution and use in source and binary forms, with or without modification, are 
% permitted provided that the following conditions are met: 
% 
% 1. Redistributions of source code must retain the above copyright notice, this list of 
% conditions and the following disclaimer. 
%
% 2. Redistributions in binary form must reproduce the above copyright notice, this list
% of conditions and the following disclaimer in the documentation and/or other materials % provided with the distribution. 
% 
% THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDER, WILLIAM SMITH ``AS IS'' AND ANY EXPRESS 
% OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF 
% MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL 
% THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, 
% SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT 
% OF SUBSTITUTE GOODS ORSERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) 
% HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, 
% OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS 
% SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.s 

%% Code
periods = floor(t / deltat); 
S = zeros(periods, 1); 
S(1) = S0; 

exp_minus_lambda_deltat = exp(-lambda*deltat);

% Calculate the random term. 
if (lambda == 0) 
    % Handle the case of lambda = 0 i.e. no mean reversion. 
    dWt = sqrt(deltat) * randn(periods,1); 
else
    dWt = sqrt((1-exp(-2*lambda* deltat))/(2*lambda)) * randn(periods,1); 
end

% And iterate through time calculating each price. 

for t=2:1:periods 
    S(t) = S(t-1)*exp_minus_lambda_deltat + mu*(1-exp_minus_lambda_deltat) + sigma*dWt(t); 
end

% OPTIM Note : % Precalculating all dWt's rather than one-per loop makes this function 
% approx 50% faster. Useful for Monte-Carlo simulations. 

% OPTIM Note : calculating exp(-lambda*deltat) makes it roughly 50% faster 
% again. 

% OPTIM Note : this is only about 25% slower than the rough calculation 
% without the exp correction. 

end

In [ ]:

%simul_vega_test_cms_spread_ra_lgm2f_20180828_vegaMap.m
clc
clear
valueDate = '2018-08-27';
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.001;
%shift size 10bp
volShiftAmount = 0.001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);



% vegaSimulationType = 'ParallelVega';
vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';

if strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999959 
    0.25	0.995904 
    0.5	0.991447 
    0.75	0.986791 
    1	0.982157 
    2	0.963331 
    3	0.944487 
    4	0.925354 
    5	0.905952 
    7	0.868350 
    10	0.811785 
    12	0.774859 
    15	0.725355 
    20	0.650550 
    25	0.584638 
    30	0.525482 
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve


    rawData = [0.002739726	0.999959 
    0.25	0.996135 
    0.5	0.991903 
    0.75	0.987432 
    1	0.982765 
    2	0.963734 
    3	0.943126 
    5	0.897282 
    10	0.789258 
    20	0.628672 
    30	0.503774 
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

    %JaeYeol
    dHTest = [0.060438773	0.585467789
    0.064928789	0.621766986
    0.059045192	0.705355899
    0.070178721	0.6972879
    0.084746775	0.641430112
    0.079080439	0.601700141
    0.1039792	0.504405369
    0.129262749	0.522031261
    0.14309303	0.436403222
    0.167840108	0.341211636
    0.171134327	0.384814515
    0.17295042	0.333229804
    0.205446544	0.253016302
    0.190785578	0.233852044
    0.212042588	0.235092969
    0.208563343	0.261106368
    0.26724335	0.213909029
    0.24410813	0.131731127
    0.292886806	0.110481688
    0.302159283	0.107800708
    0.341084552	0.095566454
    0.353944447	0.072422244
    0.421105867	0.033562964
    0.357823112	0.006568462
    ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

    %jaeweol
    AlphaTest = [0.032529518	0.005904921
    0.021882297	0.0061619
    0.015498414	0.006184972
    0.011471555	0.007705108
    0.008105853	0.008915352
    0.010349202	0.008306695
    0.006993193	0.012332999
    0.000309834	0.020072884
    ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,false);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
    closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
        closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
%simul_vega_test_cms_spread_ra_lgm2f_20180828_newParams0816_ycIn.m

clc
clear
valueDate = '2018-08-27';
dateH = H_Date(valueDate);
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.001;
%shift size 10bp
volShiftAmount = 0.001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);



% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    dataType= {'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenor = {'1D'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '4Y'
              '5Y'
              '7Y'
              '10Y'
              '12Y'
              '15Y'
              '20Y'
              '25Y'
              '30Y'
    };

    dataRate = {0.0148
    0.0165
    0.01725
    0.017775
    0.01805
    0.018725
    0.019075
    0.019425
    0.019775
    0.020175
    0.020825
    0.0212
    0.02135
    0.02145
    0.02145
    0.02145
    };

    dataIn.dataType = dataType;
    dataIn.dataTenor = dataTenor;
    dataIn.dataRate = dataRate;
    
    curveOut = YieldCurveBootStrap('KRW',dateH,dataIn);
    
    rawData = curveOut(:,1:2);
%     rawData = [0.002739726	0.999959 
%     0.25	0.995904 
%     0.5	0.991447 
%     0.75	0.986791 
%     1	0.982157 
%     2	0.963331 
%     3	0.944487 
%     4	0.925354 
%     5	0.905952 
%     7	0.868350 
%     10	0.811785 
%     12	0.774859 
%     15	0.725355 
%     20	0.650550 
%     25	0.584638 
%     30	0.525482 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve


    rawData = [0.002739726	0.999959 
    0.25	0.996135 
    0.5	0.991903 
    0.75	0.987432 
    1	0.982765 
    2	0.963734 
    3	0.943126 
    5	0.897282 
    10	0.789258 
    20	0.628672 
    30	0.503774 
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
%simul_vega_test_cms_spread_ra_lgm2f_20180828_newParams0816.m

clc
clear
valueDate = '2018-08-27';
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.001;
%shift size 10bp
volShiftAmount = 0.001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);



% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    curveOut = YieldCurveBootStrap('KRW',dateH,dataIn)
    
    rawData = [0.002739726	0.999959 
    0.25	0.995904 
    0.5	0.991447 
    0.75	0.986791 
    1	0.982157 
    2	0.963331 
    3	0.944487 
    4	0.925354 
    5	0.905952 
    7	0.868350 
    10	0.811785 
    12	0.774859 
    15	0.725355 
    20	0.650550 
    25	0.584638 
    30	0.525482 
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve


    rawData = [0.002739726	0.999959 
    0.25	0.996135 
    0.5	0.991903 
    0.75	0.987432 
    1	0.982765 
    2	0.963734 
    3	0.943126 
    5	0.897282 
    10	0.789258 
    20	0.628672 
    30	0.503774 
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
%simul_vega_test_cms_spread_ra_lgm2f_20180828.m

clc
clear
valueDate = '2018-08-27';
valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.001;
%shift size 10bp
volShiftAmount = 0.001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);



vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';

if strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999959 
    0.25	0.995904 
    0.5	0.991447 
    0.75	0.986791 
    1	0.982157 
    2	0.963331 
    3	0.944487 
    4	0.925354 
    5	0.905952 
    7	0.868350 
    10	0.811785 
    12	0.774859 
    15	0.725355 
    20	0.650550 
    25	0.584638 
    30	0.525482 
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve


    rawData = [0.002739726	0.999959 
    0.25	0.996135 
    0.5	0.991903 
    0.75	0.987432 
    1	0.982765 
    2	0.963734 
    3	0.943126 
    5	0.897282 
    10	0.789258 
    20	0.628672 
    30	0.503774 
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

    %JaeYeol
    dHTest = [0.060438773	0.585467789
    0.064928789	0.621766986
    0.059045192	0.705355899
    0.070178721	0.6972879
    0.084746775	0.641430112
    0.079080439	0.601700141
    0.1039792	0.504405369
    0.129262749	0.522031261
    0.14309303	0.436403222
    0.167840108	0.341211636
    0.171134327	0.384814515
    0.17295042	0.333229804
    0.205446544	0.253016302
    0.190785578	0.233852044
    0.212042588	0.235092969
    0.208563343	0.261106368
    0.26724335	0.213909029
    0.24410813	0.131731127
    0.292886806	0.110481688
    0.302159283	0.107800708
    0.341084552	0.095566454
    0.353944447	0.072422244
    0.421105867	0.033562964
    0.357823112	0.006568462
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

    %jaeweol
    AlphaTest = [0.032529518	0.005904921
    0.021882297	0.0061619
    0.015498414	0.006184972
    0.011471555	0.007705108
    0.008105853	0.008915352
    0.010349202	0.008306695
    0.006993193	0.012332999
    0.000309834	0.020072884
    ];


    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end




In [ ]:
%simul_riskyFixedBond_HazardRateBootStrapping_ytmDelta.m

clc
clear
valueDate = '2018-09-11';

dateH0 = H_Date(valueDate);

dateH = H_Date(valueDate);


maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
curveShiftAmount = 0.0001;

simulNPV = zeros(100,1);
simulfairRate = zeros(100,1);


% deltaSimulationType = 'ParallelDelta';
deltaSimulationType = 'PointByPointDelta';

% deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';
% deltaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*4+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
else
    idxMax = 2*1+1;
end

%% import and construct a hazardRateCurve
% 
% 
% dataType= {'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% };
% 
dataType= {'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
};

dataTenor = {'1Y'
'2Y'
'3Y'
'5Y'
};
% 
% % % 우리은행 AAA
% % dataRate = {0.01926
% % 0.02043
% % 0.02129
% % 0.02301
% % };
% 
% % % % 신한카드 AA+
% % dataRate = {0.02039
% % 0.02199
% % 0.02302
% % 0.02507
% % };
% 
% % 벤츠 파이넨셜 A+
dataRate = {0.02082
0.02394
0.02849
0.03151
};
% 
% % dataRate = {0.02978608428583750000
% % 0.02987540501789370000
% % 0.02989337047641950000
% % };

% market data IN
dataIn = {};
dataIn.dataType = dataType;
dataIn.dataTenor = dataTenor;
dataIn.dataRate = dataRate;
    

% other static setting for bootstrapping

riskyFixedBondParams = {};
riskyFixedBondParams.recoveryRate = 0.4;

%% import and construct a discountCurve
curveName = 'KRW_KTB';
ycDiscData = GetYieldCurveFromOctagon(valueDate,curveName);
ycDiscDataOrig = ycDiscData;

yieldCurveParamsDisc = {};
 %KTB
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';

targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];

yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;

for idxSimul =1:idxMax
    
    dataRateOrig = dataRate;
    dataRateSim = dataRateOrig;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if idxSimul == 2

                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                end

            elseif idxSimul ==3
                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if mod(idxSimul,2) == 0
                idx2 = idxSimul/2;
                dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idxSimul,2) == 1
                idx2 = (idxSimul-1)/2;
                dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
            end
        end
    end

    dataIn.dataRate = dataRateSim;
    
    
    dataRateOrig = ycDiscDataOrig.dataRate;
    dataRateSim = dataRateOrig;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if idxSimul == 2

                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                end

            elseif idxSimul ==3
                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if mod(idxSimul,2) == 0
                idx2 = idxSimul/2;
                dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idxSimul,2) == 1
                idx2 = (idxSimul-1)/2;
                dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    ycDiscData.dataRate = dataRateSim;
    
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycDiscData,yieldCurveParamsDisc);
    
    rawDataDisc = curveOutDisc(:,1:2);

    for idx=1:size(rawDataDisc,1)
        rawDataDisc(idx,2) = -log(rawDataDisc(idx,2))/rawDataDisc(idx,1);
    end

    % hazardRateCurveParams : bootStrapping input arg
    hazardRateCurveParams = {};
    hazardRateCurveParams.recoveryRate = 0.4;

    mktDataDisc = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataDisc,dateH,'KTB','discountZero'});
    discountCurveMktData = MktData(mktDataDisc);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    hazardRateCurveParams.discountCurve=discountCurve;

    riskyFixedBondParams.discountCurve=discountCurve;

% % survProbCurve
% rawData = [0.002739726	0.999972603
% 0.252054795	0.997503122
% ];
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
% {rawData,dateH,'KRW','KRWSurvProb'});
% survProbCurveMktData = MktData(mktData);
% survProbCurveMktData.params('convention') = {};
% survProbCurve =  SurvProbCurve(survProbCurveMktData);

%for riskyFixedBond 
hazardRateCurveParams.settlesAccrual = false;
hazardRateCurveParams.paysAtDefaultTime = false;

hazardRateCurveParams.engineType = 'Integrate';
% hazardRateCurveParams.engineType = 'Mid';

%riskyFixedBond settings
hazardRateCurveParams.riskyFixedBondSettlementDays = 1;
hazardRateCurveParams.riskyFixedBondBasis = 'Act365';
hazardRateCurveParams.riskyFixedBondConvention = 'ModifiedFollowing';
hazardRateCurveParams.riskyFixedBondPeriod = '3M';
hazardRateCurveParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

targetTimeBucket = [ 0.0027397 
];

hazardRateCurveParams.targetTimeBucket = targetTimeBucket;
curveOut = HazardRateCurveBootStrap('KRW',dateH,dataIn,hazardRateCurveParams);

% survProbCurve
rawDataSurv = zeros(length(curveOut.survProb),2);
rawDataSurv(:,1) = curveOut.times(:);
rawDataSurv(:,2) = curveOut.survProb(:);

% rawDataSurv = [1.01369863	0.993992919
% 2.010958904	0.98113753
% 3.008219178	0.953832109
% 5.005479452	0.913660385
% ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
{rawDataSurv,dateH,'KRW','KRWSurvProb'});
survProbCurveMktData = MktData(mktData);
survProbCurveMktData.params('convention') = {};
survProbCurve =  SurvProbCurve(survProbCurveMktData);

riskyFixedBondParams.survProbCurve = survProbCurve;
%riskyFixedBondParams settings
riskyFixedBondParams.riskyFixedBondSettlementDays = 1;
riskyFixedBondParams.riskyFixedBondBasis = 'Act365';
riskyFixedBondParams.riskyFixedBondConvention = 'ModifiedFollowing';
riskyFixedBondParams.riskyFixedBondPeriod = '3M';
% riskyFixedBondParams.spreadCDSDateGenerationRule = 'TwentiethIMM';
% riskyFixedBondParams.spreadCDSDateGenerationRule = 'BackwardCDS';
riskyFixedBondParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

riskyFixedBondParams.settlesAccrual = false;
riskyFixedBondParams.paysAtDefaultTime = false;

riskyFixedBondParams.engineType = 'Integrate';
% riskyFixedBondParams.engineType = 'Mid';

riskyFixedBondParams.evaluationDate = riskyFixedBondParams.discountCurve.asOfDate;

riskyFixedBondParams.efffectiveStartDate = H_Date('2018-09-12');

riskyFixedBondParams.startDate = riskyFixedBondParams.efffectiveStartDate;
riskyFixedBondParams.endDate = AddTenor(riskyFixedBondParams.efffectiveStartDate,'5Y');
jointCalendar = MakeCalendar(dateH,{},'30Y');
riskyFixedBondParams.schedule = Schedule(riskyFixedBondParams.startDate,riskyFixedBondParams.endDate,riskyFixedBondParams.riskyFixedBondPeriod,jointCalendar, riskyFixedBondParams.riskyFixedBondConvention,...
                              riskyFixedBondParams.riskyFixedBondDateGenerationRule,false);

riskyFixedBondParams.type = 'Buyer';  
riskyFixedBondParams.nominal = 10000.0;
riskyFixedBondParams.premiumRate = 0.03151;

riskyFixedBond = RiskyFixedBond(riskyFixedBondParams.type,riskyFixedBondParams.engineType,riskyFixedBondParams.nominal,...
                                riskyFixedBondParams.schedule,riskyFixedBondParams.premiumRate,riskyFixedBondParams.riskyFixedBondBasis,...
                                riskyFixedBondParams.riskyFixedBondConvention,riskyFixedBondParams.settlesAccrual,riskyFixedBondParams.paysAtDefaultTime,...
                                riskyFixedBondParams.efffectiveStartDate);
                            

outInfo = riskyFixedBond.Calculate(riskyFixedBondParams.recoveryRate,riskyFixedBondParams.survProbCurve,riskyFixedBondParams.discountCurve);

simulNPV(idxSimul) = outInfo.NPV;
simulfairRate(idxSimul) = outInfo.fairRate;


end

aaa = 1.0;
% %crossCheck
% dataDiff = zeros(length(dataRate),1);
% 
% if strcmp(dataType{1},'RiskyFixedBond')
% 
%     dataType= {'SpreadCDS'
%     'SpreadCDS'
%     'SpreadCDS'
%     'SpreadCDS'
%     };
%     
%     dataRateOrg = dataRate;
%     
%     for i=1:length(dataRate)
%         dataRate{i} = curveOut.impliedParCDSSPread(i);
%     end
%     
%     dataIn.dataType = dataType;
%     dataIn.dataRate = dataRate;
% 
%     curveOutCrossCheck = HazardRateCurveBootStrap('KRW',dateH,dataIn,riskyFixedBondParams);
%     
%     dataRateNew = dataRateOrg;
%     
%     for i=1:length(dataRate)
%         dataRateNew{i} = curveOutCrossCheck.impliedYTM(i);
%         dataDiff(i) = dataRateNew{i} - dataRateOrg{i}; 
%  
%     end
%     
% else
%     
%     dataType= {'RiskyFixedBond'
%     'RiskyFixedBond'
%     'RiskyFixedBond'
%     'RiskyFixedBond'
%     };
% 
%     dataRateOrg = dataRate;
%     
%     for i=1:length(dataRate)
%         dataRate{i} = curveOut.impliedYTM(i);
%     end
%     
%     dataIn.dataType = dataType;
%     dataIn.dataRate = dataRate;
%     
%     curveOutCrossCheck = HazardRateCurveBootStrap('KRW',dateH,dataIn,riskyFixedBondParams);
%     
%     dataRateNew = dataRateOrg;
%     
%     for i=1:length(dataRate)
%         dataRateNew{i} = curveOutCrossCheck.impliedParCDSSPread(i);
%         dataDiff(i) = dataRateNew{i} - dataRateOrg{i}; 
%     end
%     
% end


aaa =1.0;


In [ ]:
%simul_riskyFixedBond_HazardRateBootStrapping_ktbDelta_20211019.m

clc
clear
valueDate = '2018-09-11';

dateH0 = H_Date(valueDate);

dateH = H_Date(valueDate);


maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
curveShiftAmount = 0.0001;

simulNPV = zeros(100,1);
simulfairRate = zeros(100,1);


% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';

deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';
% deltaSimulationType = 'None';

idxMax = 1;

shiftAmountLow = -0.001;
if strcmp(deltaSimulationType,'ParallelDelta')
%     idxMax = 2*1+1;
    idxMax = 21;
    
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*4+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
%     idxMax = 2*1+1;

    idxMax = 21;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
else
    idxMax = 2*1+1;
end

%% import and construct a hazardRateCurve
% 
% 
% dataType= {'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% };
% 
dataType= {'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
};

dataTenor = {'1Y'
'2Y'
'3Y'
'5Y'
};
% 
% % % 우리은행 AAA
% % dataRate = {0.01926
% % 0.02043
% % 0.02129
% % 0.02301
% % };
% 
% % % % 신한카드 AA+
% % dataRate = {0.02039
% % 0.02199
% % 0.02302
% % 0.02507
% % };
% 
% % 벤츠 파이넨셜 A+
dataRate = {0.02082
0.02394
0.02849
0.03151
};
% 
% % dataRate = {0.02978608428583750000
% % 0.02987540501789370000
% % 0.02989337047641950000
% % };

% market data IN
dataIn = {};
dataIn.dataType = dataType;
dataIn.dataTenor = dataTenor;
dataIn.dataRate = dataRate;
    

% other static setting for bootstrapping

riskyFixedBondParams = {};
% riskyFixedBondParams.recoveryRate = 0.4;
riskyFixedBondParams.recoveryRate = 0.0;


%% MarketData YLD
%% calendar
% global dictionary(map) init
globalDict = containers.Map();

calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDate,calendarNames,'30Y');

globalDict('jointCalendar') = jointCalendar;
%yldCurveSetting
yiledCurveSettings = containers.Map();

%KRW_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;1.5;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%% import and construct a discountCurve
curveName = 'KRW_KTB';
ycDiscData = GetYieldCurveFromOctagon(valueDate,curveName,globalDict);
ycDiscDataOrig = ycDiscData;

yieldCurveParamsDisc = {};
 %KTB
 yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';

targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];

yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;

for idxSimul =1:idxMax
    
    dataRateOrig = dataRate;
    dataRateSim = dataRateOrig;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta
        dataRateSim = dataRateOrig;
        
        
        for idx2= 1: length(dataRateOrig)
            dataRateSim{idx2} = dataRateOrig{idx2} + shiftAmountLow + curveShiftAmount*1.0*(idxSimul-1);
        end

%         if idxSimul==1
%             dataRateSim = dataRateOrig;
%         else
%             if idxSimul == 2
% 
%                 for idx2= 1: length(dataRateOrig)
%                     dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
%                 end
% 
%             elseif idxSimul ==3
%                 for idx2= 1: length(dataRateOrig)
%                     dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
%                 end
%             end
%         end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if mod(idxSimul,2) == 0
                idx2 = idxSimul/2;
                dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idxSimul,2) == 1
                idx2 = (idxSimul-1)/2;
                dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
            end
        end
    end

    dataIn.dataRate = dataRateSim;
    
   %% KTB Curve Shift
    
    dataRateOrig = ycDiscDataOrig.ycData.dataRate;
    dataRateSim = dataRateOrig;
    
%     if strcmp(deltaSimulationType,'DiscParallelDelta')   
%     
% %% YieldCurve simulation by parallel translation for delta
%         dataRateSim = dataRateOrig;
%         
%         
%         for idx2= 1: length(dataRateOrig)
%             dataRateSim{idx2} = dataRateOrig{idx2} + shiftAmountLow + curveShiftAmount*1.0*(idxSimul-1);
%         end
%         
% %         if idxSimul==1
% %             dataRateSim = dataRateOrig;
% %         else
% %             if idxSimul == 2
% % 
% %                 for idx2= 1: length(dataRateOrig)
% %                     dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
% %                 end
% % 
% %             elseif idxSimul ==3
% %                 for idx2= 1: length(dataRateOrig)
% %                     dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
% %                 end
% %             end
% %         end
%         
%     elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
%         if idxSimul==1
%             dataRateSim = dataRateOrig;
%         else
%             if mod(idxSimul,2) == 0
%                 idx2 = idxSimul/2;
%                 dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
%                 
%             elseif mod(idxSimul,2) == 1
%                 idx2 = (idxSimul-1)/2;
%                 dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
%             end
%         end
%     end
    
    ycDiscData.ycData.dataRate = dataRateSim;
    
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycDiscData.ycData,yieldCurveParamsDisc);
    
%     rawDataDisc = curveOutDisc(:,1:2);
    rawDataDisc = curveOutDisc.bootStrapOut.forwardCurve.params('rawData');

%     for idx=1:size(rawDataDisc,1)
%         rawDataDisc(idx,2) = -log(rawDataDisc(idx,2))/rawDataDisc(idx,1);
%     end

    % hazardRateCurveParams : bootStrapping input arg
    hazardRateCurveParams = {};
%     hazardRateCurveParams.recoveryRate = 0.4;
    hazardRateCurveParams.recoveryRate = 0.0;
    

    mktDataDisc = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataDisc,dateH,'KTB','discountZero'});
    discountCurveMktData = MktData(mktDataDisc);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    hazardRateCurveParams.discountCurve=discountCurve;

    riskyFixedBondParams.discountCurve=discountCurve;

% % survProbCurve
% rawData = [0.002739726	0.999972603
% 0.252054795	0.997503122
% ];
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
% {rawData,dateH,'KRW','KRWSurvProb'});
% survProbCurveMktData = MktData(mktData);
% survProbCurveMktData.params('convention') = {};
% survProbCurve =  SurvProbCurve(survProbCurveMktData);

%for riskyFixedBond 
hazardRateCurveParams.settlesAccrual = false;
hazardRateCurveParams.paysAtDefaultTime = false;

hazardRateCurveParams.engineType = 'Integrate';
% hazardRateCurveParams.engineType = 'Mid';

%riskyFixedBond settings
hazardRateCurveParams.riskyFixedBondSettlementDays = 1;
hazardRateCurveParams.riskyFixedBondBasis = 'Act365';
hazardRateCurveParams.riskyFixedBondConvention = 'ModifiedFollowing';
hazardRateCurveParams.riskyFixedBondPeriod = '3M';
hazardRateCurveParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

targetTimeBucket = [ 0.0027397 
];

hazardRateCurveParams.targetTimeBucket = targetTimeBucket;
curveOut = HazardRateCurveBootStrap('KRW',dateH,dataIn,hazardRateCurveParams);

% survProbCurve
rawDataSurv = zeros(length(curveOut.survProb),2);
rawDataSurv(:,1) = curveOut.times(:);
rawDataSurv(:,2) = curveOut.survProb(:);

% rawDataSurv = [1.01369863	0.993992919
% 2.010958904	0.98113753
% 3.008219178	0.953832109
% 5.005479452	0.913660385
% ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
{rawDataSurv,dateH,'KRW','KRWSurvProb'});
survProbCurveMktData = MktData(mktData);
survProbCurveMktData.params('convention') = {};
survProbCurve =  SurvProbCurve(survProbCurveMktData);

riskyFixedBondParams.survProbCurve = survProbCurve;

%% to use fixed hazard rate(surProb), we shift KTB curve here
%% KTB Curve Shift
    
    dataRateOrig = ycDiscDataOrig.ycData.dataRate;
    dataRateSim = dataRateOrig;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta
        dataRateSim = dataRateOrig;
        
        
        for idx2= 1: length(dataRateOrig)
            dataRateSim{idx2} = dataRateOrig{idx2} + shiftAmountLow + curveShiftAmount*1.0*(idxSimul-1);
        end
        
%         if idxSimul==1
%             dataRateSim = dataRateOrig;
%         else
%             if idxSimul == 2
% 
%                 for idx2= 1: length(dataRateOrig)
%                     dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
%                 end
% 
%             elseif idxSimul ==3
%                 for idx2= 1: length(dataRateOrig)
%                     dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
%                 end
%             end
%         end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if mod(idxSimul,2) == 0
                idx2 = idxSimul/2;
                dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idxSimul,2) == 1
                idx2 = (idxSimul-1)/2;
                dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    ycDiscData.ycData.dataRate = dataRateSim;
    
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycDiscData.ycData,yieldCurveParamsDisc);
    
%     rawDataDisc = curveOutDisc(:,1:2);
    rawDataDisc = curveOutDisc.bootStrapOut.forwardCurve.params('rawData');

    mktDataDisc = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataDisc,dateH,'KTB','discountZero'});
    discountCurveMktData = MktData(mktDataDisc);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    riskyFixedBondParams.discountCurve=discountCurve;

%riskyFixedBondParams settings
riskyFixedBondParams.riskyFixedBondSettlementDays = 1;
riskyFixedBondParams.riskyFixedBondBasis = 'Act365';
riskyFixedBondParams.riskyFixedBondConvention = 'ModifiedFollowing';
riskyFixedBondParams.riskyFixedBondPeriod = '3M';
% riskyFixedBondParams.spreadCDSDateGenerationRule = 'TwentiethIMM';
% riskyFixedBondParams.spreadCDSDateGenerationRule = 'BackwardCDS';
riskyFixedBondParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

riskyFixedBondParams.settlesAccrual = false;
riskyFixedBondParams.paysAtDefaultTime = false;

riskyFixedBondParams.engineType = 'Integrate';
% riskyFixedBondParams.engineType = 'Mid';

riskyFixedBondParams.evaluationDate = riskyFixedBondParams.discountCurve.asOfDate;

riskyFixedBondParams.efffectiveStartDate = H_Date('2018-09-12');

riskyFixedBondParams.startDate = riskyFixedBondParams.efffectiveStartDate;
riskyFixedBondParams.endDate = AddTenor(riskyFixedBondParams.efffectiveStartDate,'5Y');
jointCalendar = MakeCalendar(dateH,{},'30Y');
riskyFixedBondParams.schedule = Schedule(riskyFixedBondParams.startDate,riskyFixedBondParams.endDate,riskyFixedBondParams.riskyFixedBondPeriod,jointCalendar, riskyFixedBondParams.riskyFixedBondConvention,...
                              riskyFixedBondParams.riskyFixedBondDateGenerationRule,false);

riskyFixedBondParams.type = 'Buyer';  
riskyFixedBondParams.nominal = 10000.0;
riskyFixedBondParams.premiumRate = 0.03151;

riskyFixedBond = RiskyFixedBond(riskyFixedBondParams.type,riskyFixedBondParams.engineType,riskyFixedBondParams.nominal,...
                                riskyFixedBondParams.schedule,riskyFixedBondParams.premiumRate,riskyFixedBondParams.riskyFixedBondBasis,...
                                riskyFixedBondParams.riskyFixedBondConvention,riskyFixedBondParams.settlesAccrual,riskyFixedBondParams.paysAtDefaultTime,...
                                riskyFixedBondParams.efffectiveStartDate);
                            

outInfo = riskyFixedBond.Calculate(riskyFixedBondParams.recoveryRate,riskyFixedBondParams.survProbCurve,riskyFixedBondParams.discountCurve);

simulNPV(idxSimul) = outInfo.NPV;
simulfairRate(idxSimul) = outInfo.fairRate;


end

aaa = 1.0;
% %crossCheck
% dataDiff = zeros(length(dataRate),1);
% 
% if strcmp(dataType{1},'RiskyFixedBond')
% 
%     dataType= {'SpreadCDS'
%     'SpreadCDS'
%     'SpreadCDS'
%     'SpreadCDS'
%     };
%     
%     dataRateOrg = dataRate;
%     
%     for i=1:length(dataRate)
%         dataRate{i} = curveOut.impliedParCDSSPread(i);
%     end
%     
%     dataIn.dataType = dataType;
%     dataIn.dataRate = dataRate;
% 
%     curveOutCrossCheck = HazardRateCurveBootStrap('KRW',dateH,dataIn,riskyFixedBondParams);
%     
%     dataRateNew = dataRateOrg;
%     
%     for i=1:length(dataRate)
%         dataRateNew{i} = curveOutCrossCheck.impliedYTM(i);
%         dataDiff(i) = dataRateNew{i} - dataRateOrg{i}; 
%  
%     end
%     
% else
%     
%     dataType= {'RiskyFixedBond'
%     'RiskyFixedBond'
%     'RiskyFixedBond'
%     'RiskyFixedBond'
%     };
% 
%     dataRateOrg = dataRate;
%     
%     for i=1:length(dataRate)
%         dataRate{i} = curveOut.impliedYTM(i);
%     end
%     
%     dataIn.dataType = dataType;
%     dataIn.dataRate = dataRate;
%     
%     curveOutCrossCheck = HazardRateCurveBootStrap('KRW',dateH,dataIn,riskyFixedBondParams);
%     
%     dataRateNew = dataRateOrg;
%     
%     for i=1:length(dataRate)
%         dataRateNew{i} = curveOutCrossCheck.impliedParCDSSPread(i);
%         dataDiff(i) = dataRateNew{i} - dataRateOrg{i}; 
%     end
%     
% end


aaa =1.0;


In [ ]:
%simul_delta_test_cms_spread_ra_lgm2f_20190523_yc_octagon.m

clc
clear
valueDate = '2019-05-22';
dateH = H_Date(valueDate);
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
volShiftAmount = 0.001;
curveShiftAmount = 0.0001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';
deltaSimulationType = 'None';

% deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';

% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*16+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
elseif strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

globalDict = containers.Map();

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    curveName = 'KRW_IRS';
    bootStrapFlag = true;
    ycData = GetYieldCurveFromOctagon(valueDate,curveName,globalDict,bootStrapFlag);

    dataRate = ycData.dataRate;
    dataRateSim = dataRate;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateSim = dataRate;
        else
            if idx1 == 2

                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idx1==1
            dataRateSim = dataRate;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
%     dataIn.dataType = dataType;
%     dataIn.dataTenor = dataTenor;
    % important !!
    
%     dataIn.dataRate = dataRateSim;
    ycData.dataRate = dataRateSim;
    
    yieldCurveParams = {};
    yieldCurveParams.floatingLegTenor = '3M';
    yieldCurveParams.floatingLegDayCounter = 'Act365';
    yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
    yieldCurveParams.fixingDays = 1;
    
    yieldCurveParams.fixedLegTenor = '3M';
    yieldCurveParams.fixedLegDayCounter = 'Act365';
    yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
   
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
4
5
7
10
12
15
20
25
30];
    yieldCurveParams.targetTimeBucket = targetTimeBucket;
    curveOut = YieldCurveBootStrap('KRW',dateH,ycData,yieldCurveParams);
    
    rawData = curveOut(:,1:2);

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    curveName = 'KRW_KTB';
    ycData = GetYieldCurveFromOctagon(valueDate,curveName);
   

    dataRateDisc = ycData.dataRate;
    dataRateDiscSim = dataRateDisc;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if idx1 == 2

                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
%     dataIn.dataType = dataTypeDisc;
%     dataIn.dataTenor = dataTenorDisc;
%     % important !!
%     
%     dataIn.dataRate = dataRateDiscSim;
    ycData.dataRate = dataRateDiscSim;
    
    yieldCurveParamsDisc = {};
    
    yieldCurveParamsDisc.floatingLegTenor = '3M';
    yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
    yieldCurveParamsDisc.floatingLegConvention = 'Following';
    yieldCurveParamsDisc.fixingDays = 1;
    
    yieldCurveParamsDisc.fixedLegTenor = '6M';
    yieldCurveParamsDisc.fixedLegDayCounter = '30360';
    yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
    
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];
    yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycData,yieldCurveParamsDisc);
    
    rawData = curveOutDisc(:,1:2);

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    
    %0831
%     rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
%     1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
%     2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
%     3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
%     4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
%     5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
%     7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
%     10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
%     15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
%     ];

    %0827
%     rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
%     1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
%     2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
%     3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
%     4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
%     5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
%     7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
%     10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
%     15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
%     ];

    %0425
    surfName = '032111_SWAPT';
%     swaptionVolData = GetYieldCurveFromOctagon(valueDate,surfName);
    swaptionVolData = GetSwaptionVolSurfaceFromOctagon(valueDate,surfName);
    rawDataSwaption = swaptionVolData.rawData;
    
    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
%simul_delta_test_cms_spread_ra_lgm2f_20180831_yc_octagon_usd.m

clc
clear
valueDate = '2018-08-27';
dateH = H_Date(valueDate);
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
volShiftAmount = 0.001;
curveShiftAmount = 0.0001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';
deltaSimulationType = 'None';

% deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';

% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*16+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
elseif strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    curveName = 'USD_IRS';
    ycData = GetYieldCurveFromOctagon(valueDate,curveName);

    dataRate = ycData.dataRate;
    dataRateSim = dataRate;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateSim = dataRate;
        else
            if idx1 == 2

                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idx1==1
            dataRateSim = dataRate;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
%     dataIn.dataType = dataType;
%     dataIn.dataTenor = dataTenor;
    % important !!
    
%     dataIn.dataRate = dataRateSim;
    ycData.dataRate = dataRateSim;
    
    yieldCurveParams = {};
    yieldCurveParams.floatingLegTenor = '3M';
    yieldCurveParams.floatingLegDayCounter = 'Act360';
    yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
    yieldCurveParams.fixingDays = 2;
    
    yieldCurveParams.fixedLegTenor = '6M';
    yieldCurveParams.fixedLegDayCounter = '30360';
    yieldCurveParams.fixedLegConvention = 'Following';
    targetTimeBucket = [0.002739726
0.083333333
0.166666667
0.25
0.5
1
2
3
4
5
7
10
12
15
20
25
30
];
    yieldCurveParams.targetTimeBucket = targetTimeBucket;
    curveOut = YieldCurveBootStrap('KRW',dateH,ycData,yieldCurveParams);
    
    rawData = curveOut(:,1:2);
%     rawData = [0.002739726	0.999959 
%     0.25	0.995904 
%     0.5	0.991447 
%     0.75	0.986791 
%     1	0.982157 
%     2	0.963331 
%     3	0.944487 
%     4	0.925354 
%     5	0.905952 
%     7	0.868350 
%     10	0.811785 
%     12	0.774859 
%     15	0.725355 
%     20	0.650550 
%     25	0.584638 
%     30	0.525482 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    curveName = 'KRW_KTB';
    ycData = GetYieldCurveFromOctagon(valueDate,curveName);
    %     dataTypeDisc= {'Deposit'
%     'Deposit'
%     'Deposit'
%     'Deposit'
%     'Swap'
%     'Swap'
%     'Swap'
%     'Swap'
%     'Swap'
%     'Swap'
%     'Swap'
%     };
% 
%     dataTenorDisc = {'1D'
%               '3M'
%               '6M'
%               '9M'
%               '1Y'
%               '2Y'
%               '3Y'
%               '5Y'
%               '10Y'
%               '20Y'
%               '30Y'
%     };
% 
%     dataRateDisc = {0.014800 
% 0.015540 
% 0.016330 
% 0.016980 
% 0.017470 
% 0.018550 
% 0.019590 
% 0.021720 
% 0.023640 
% 0.023320 
% 0.023050 
%     };

    dataRateDisc = ycData.dataRate;
    dataRateDiscSim = dataRateDisc;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if idx1 == 2

                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
%     dataIn.dataType = dataTypeDisc;
%     dataIn.dataTenor = dataTenorDisc;
%     % important !!
%     
%     dataIn.dataRate = dataRateDiscSim;
    ycData.dataRate = dataRateDiscSim;
    
    yieldCurveParamsDisc = {};
    yieldCurveParamsDisc.fixedLegTenor = '6M';
    yieldCurveParamsDisc.fixedLegDayCounter = '30360';
%     yieldCurveParamsDisc.fixedLegDayCounter = 'Act365';
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];
    yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycData,yieldCurveParamsDisc);
    
    rawData = curveOutDisc(:,1:2);

%     rawData = [0.002739726	0.999959 
%     0.25	0.996135 
%     0.5	0.991903 
%     0.75	0.987432 
%     1	0.982765 
%     2	0.963734 
%     3	0.943126 
%     5	0.897282 
%     10	0.789258 
%     20	0.628672 
%     30	0.503774 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    %0831
%     rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
%     1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
%     2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
%     3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
%     4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
%     5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
%     7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
%     10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
%     15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
%     ];

    %0827
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
%simul_delta_test_cms_spread_ra_lgm2f_20180831_yc_octagon.m

clc
clear
valueDate = '2018-08-27';
dateH = H_Date(valueDate);
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
volShiftAmount = 0.001;
curveShiftAmount = 0.0001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';
deltaSimulationType = 'None';

% deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';

% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*16+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
elseif strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    curveName = 'KRW_IRS';
    ycData = GetYieldCurveFromOctagon(valueDate,curveName);

    dataRate = ycData.dataRate;
    dataRateSim = dataRate;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateSim = dataRate;
        else
            if idx1 == 2

                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idx1==1
            dataRateSim = dataRate;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
%     dataIn.dataType = dataType;
%     dataIn.dataTenor = dataTenor;
    % important !!
    
%     dataIn.dataRate = dataRateSim;
    ycData.dataRate = dataRateSim;
    
    yieldCurveParams = {};
    yieldCurveParams.floatingLegTenor = '3M';
    yieldCurveParams.floatingLegDayCounter = 'Act365';
    yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
    yieldCurveParams.fixingDays = 1;
    
    yieldCurveParams.fixedLegTenor = '3M';
    yieldCurveParams.fixedLegDayCounter = 'Act365';
    yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
   
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
4
5
7
10
12
15
20
25
30];
    yieldCurveParams.targetTimeBucket = targetTimeBucket;
    curveOut = YieldCurveBootStrap('KRW',dateH,ycData,yieldCurveParams);
    
    rawData = curveOut(:,1:2);

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    curveName = 'KRW_KTB';
    ycData = GetYieldCurveFromOctagon(valueDate,curveName);
   

    dataRateDisc = ycData.dataRate;
    dataRateDiscSim = dataRateDisc;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if idx1 == 2

                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
%     dataIn.dataType = dataTypeDisc;
%     dataIn.dataTenor = dataTenorDisc;
%     % important !!
%     
%     dataIn.dataRate = dataRateDiscSim;
    ycData.dataRate = dataRateDiscSim;
    
    yieldCurveParamsDisc = {};
    
    yieldCurveParamsDisc.floatingLegTenor = '3M';
    yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
    yieldCurveParamsDisc.floatingLegConvention = 'Following';
    yieldCurveParamsDisc.fixingDays = 1;
    
    yieldCurveParamsDisc.fixedLegTenor = '6M';
    yieldCurveParamsDisc.fixedLegDayCounter = '30360';
    yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
    
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];
    yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycData,yieldCurveParamsDisc);
    
    rawData = curveOutDisc(:,1:2);

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    
    %0831
%     rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
%     1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
%     2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
%     3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
%     4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
%     5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
%     7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
%     10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
%     15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
%     ];

    %0827
%     rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
%     1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
%     2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
%     3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
%     4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
%     5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
%     7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
%     10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
%     15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
%     ];

    %0425
    surfName = '032111_SWAPT';
%     swaptionVolData = GetYieldCurveFromOctagon(valueDate,surfName);
    swaptionVolData = GetSwaptionVolSurfaceFromOctagon(valueDate,surfName);
    rawDataSwaption = swaptionVolData.rawData;
    
    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
% simul_delta_test_cms_spread_ra_lgm2f_20180831_newParams0816_yc.m

clc
clear
valueDate = '2018-08-27';
dateH = H_Date(valueDate);
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
volShiftAmount = 0.001;
curveShiftAmount = 0.0001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';

% deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';
deltaSimulationType = 'None';

% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*16+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
elseif strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    dataType= {'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenor = {'1D'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '4Y'
              '5Y'
              '7Y'
              '10Y'
              '12Y'
              '15Y'
              '20Y'
              '25Y'
              '30Y'
    };

    dataRate = {0.0148
    0.0165
    0.01725
    0.017775
    0.01805
    0.018725
    0.019075
    0.019425
    0.019775
    0.020175
    0.020825
    0.0212
    0.02135
    0.02145
    0.02145
    0.02145
    };

    dataRateSim = dataRate;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateSim = dataRate;
        else
            if idx1 == 2

                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idx1==1
            dataRateSim = dataRate;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataIn.dataType = dataType;
    dataIn.dataTenor = dataTenor;
    % important !!
    
    dataIn.dataRate = dataRateSim;
    
    yieldCurveParams = {};
    yieldCurveParams.floatingLegTenor = '3M';
    yieldCurveParams.floatingLegDayCounter = 'Act365';
    yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
    yieldCurveParams.fixingDays = 1;
    
    yieldCurveParams.fixedLegTenor = '3M';
    yieldCurveParams.fixedLegDayCounter = 'Act365';
    yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
   
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
4
5
7
10
12
15
20
25
30];
    yieldCurveParams.targetTimeBucket = targetTimeBucket;
    curveOut = YieldCurveBootStrap('KRW',dateH,dataIn,yieldCurveParams);
    
    rawData = curveOut(:,1:2);
%     rawData = [0.002739726	0.999959 
%     0.25	0.995904 
%     0.5	0.991447 
%     0.75	0.986791 
%     1	0.982157 
%     2	0.963331 
%     3	0.944487 
%     4	0.925354 
%     5	0.905952 
%     7	0.868350 
%     10	0.811785 
%     12	0.774859 
%     15	0.725355 
%     20	0.650550 
%     25	0.584638 
%     30	0.525482 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    dataTypeDisc= {'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenorDisc = {'1D'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '5Y'
              '10Y'
              '20Y'
              '30Y'
    };

    dataRateDisc = {0.014800 
0.015540 
0.016330 
0.016980 
0.017470 
0.018550 
0.019590 
0.021720 
0.023640 
0.023320 
0.023050 
    };

    dataRateDiscSim = dataRateDisc;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if idx1 == 2

                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataIn.dataType = dataTypeDisc;
    dataIn.dataTenor = dataTenorDisc;
    % important !!
    
    dataIn.dataRate = dataRateDiscSim;
    
    yieldCurveParamsDisc = {};
    
    yieldCurveParamsDisc.floatingLegTenor = '3M';
    yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
    yieldCurveParamsDisc.floatingLegConvention = 'Following';
    yieldCurveParamsDisc.fixingDays = 1;
    
    yieldCurveParamsDisc.fixedLegTenor = '6M';
    yieldCurveParamsDisc.fixedLegDayCounter = '30360';
    yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
    
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];
    yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,dataIn,yieldCurveParamsDisc);
    
    rawData = curveOutDisc(:,1:2);

%     rawData = [0.002739726	0.999959 
%     0.25	0.996135 
%     0.5	0.991903 
%     0.75	0.987432 
%     1	0.982765 
%     2	0.963734 
%     3	0.943126 
%     5	0.897282 
%     10	0.789258 
%     20	0.628672 
%     30	0.503774 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:


clc
clear
valueDate = '2018-08-27';
dateH = H_Date(valueDate);
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
volShiftAmount = 0.001;
curveShiftAmount = 0.0001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';

% deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';
deltaSimulationType = 'None';

% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*16+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
elseif strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    dataType= {'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenor = {'1D'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '4Y'
              '5Y'
              '7Y'
              '10Y'
              '12Y'
              '15Y'
              '20Y'
              '25Y'
              '30Y'
    };

    dataRate = {0.0148
    0.0165
    0.01725
    0.017775
    0.01805
    0.018725
    0.019075
    0.019425
    0.019775
    0.020175
    0.020825
    0.0212
    0.02135
    0.02145
    0.02145
    0.02145
    };

    dataRateSim = dataRate;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateSim = dataRate;
        else
            if idx1 == 2

                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idx1==1
            dataRateSim = dataRate;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataIn.dataType = dataType;
    dataIn.dataTenor = dataTenor;
    % important !!
    
    dataIn.dataRate = dataRateSim;
    
    yieldCurveParams = {};
    yieldCurveParams.floatingLegTenor = '3M';
    yieldCurveParams.floatingLegDayCounter = 'Act365';
    yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
    yieldCurveParams.fixingDays = 1;
    
    yieldCurveParams.fixedLegTenor = '3M';
    yieldCurveParams.fixedLegDayCounter = 'Act365';
    yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
   
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
4
5
7
10
12
15
20
25
30];
    yieldCurveParams.targetTimeBucket = targetTimeBucket;
    curveOut = YieldCurveBootStrap('KRW',dateH,dataIn,yieldCurveParams);
    
    rawData = curveOut(:,1:2);
%     rawData = [0.002739726	0.999959 
%     0.25	0.995904 
%     0.5	0.991447 
%     0.75	0.986791 
%     1	0.982157 
%     2	0.963331 
%     3	0.944487 
%     4	0.925354 
%     5	0.905952 
%     7	0.868350 
%     10	0.811785 
%     12	0.774859 
%     15	0.725355 
%     20	0.650550 
%     25	0.584638 
%     30	0.525482 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    dataTypeDisc= {'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenorDisc = {'1D'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '5Y'
              '10Y'
              '20Y'
              '30Y'
    };

    dataRateDisc = {0.014800 
0.015540 
0.016330 
0.016980 
0.017470 
0.018550 
0.019590 
0.021720 
0.023640 
0.023320 
0.023050 
    };

    dataRateDiscSim = dataRateDisc;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if idx1 == 2

                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataIn.dataType = dataTypeDisc;
    dataIn.dataTenor = dataTenorDisc;
    % important !!
    
    dataIn.dataRate = dataRateDiscSim;
    
    yieldCurveParamsDisc = {};
    
    yieldCurveParamsDisc.floatingLegTenor = '3M';
    yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
    yieldCurveParamsDisc.floatingLegConvention = 'Following';
    yieldCurveParamsDisc.fixingDays = 1;
    
    yieldCurveParamsDisc.fixedLegTenor = '6M';
    yieldCurveParamsDisc.fixedLegDayCounter = '30360';
    yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
    
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];
    yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,dataIn,yieldCurveParamsDisc);
    
    rawData = curveOutDisc(:,1:2);

%     rawData = [0.002739726	0.999959 
%     0.25	0.996135 
%     0.5	0.991903 
%     0.75	0.987432 
%     1	0.982765 
%     2	0.963734 
%     3	0.943126 
%     5	0.897282 
%     10	0.789258 
%     20	0.628672 
%     30	0.503774 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
% simul_delta_test_cms_spread_ra_lgm2f_20180831_newParams0816_usd.m

clc
clear
valueDate = '2018-08-27';
dateH = H_Date(valueDate);
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
volShiftAmount = 0.001;
curveShiftAmount = 0.0001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';

% deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';
deltaSimulationType = 'None';

% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*16+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
elseif strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    dataType= {'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenor = {'1D'
              '1M'
              '2M'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '4Y'
              '5Y'
              '7Y'
              '10Y'
              '12Y'
              '15Y'
              '20Y'
              '25Y'
              '30Y'
    };

    dataRate = {0.0191888
0.0207275
0.02196
0.0231725
0.02523
0.0258835
0.026537
0.028239
0.028679
0.028749
0.028769
0.028887
0.029219
0.029446
0.029625
0.029704
0.029595
0.029465
    };

    dataRateSim = dataRate;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateSim = dataRate;
        else
            if idx1 == 2

                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idx1==1
            dataRateSim = dataRate;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataIn.dataType = dataType;
    dataIn.dataTenor = dataTenor;
    % important !!
    
    dataIn.dataRate = dataRateSim;
    
    yieldCurveParams = {};
    yieldCurveParams.floatingLegTenor = '3M';
    yieldCurveParams.floatingLegDayCounter = 'Act360';
    yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
    yieldCurveParams.fixingDays = 2;
    
    yieldCurveParams.fixedLegTenor = '6M';
    yieldCurveParams.fixedLegDayCounter = '30360';
    yieldCurveParams.fixedLegConvention = 'Following';
%     yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
   
    targetTimeBucket = [0.002739726
0.083333333
0.166666667
0.25
0.5
1
2
3
4
5
7
10
12
15
20
25
30
];
    yieldCurveParams.targetTimeBucket = targetTimeBucket;
    curveOut = YieldCurveBootStrap('KRW',dateH,dataIn,yieldCurveParams);
    
    rawData = curveOut(:,1:2);
%     rawData = [0.002739726	0.999959 
%     0.25	0.995904 
%     0.5	0.991447 
%     0.75	0.986791 
%     1	0.982157 
%     2	0.963331 
%     3	0.944487 
%     4	0.925354 
%     5	0.905952 
%     7	0.868350 
%     10	0.811785 
%     12	0.774859 
%     15	0.725355 
%     20	0.650550 
%     25	0.584638 
%     30	0.525482 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    dataTypeDisc= {'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenorDisc = {'1D'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '5Y'
              '10Y'
              '20Y'
              '30Y'
    };

    dataRateDisc = {0.014800 
0.015540 
0.016330 
0.016980 
0.017470 
0.018550 
0.019590 
0.021720 
0.023640 
0.023320 
0.023050 
    };

    dataRateDiscSim = dataRateDisc;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if idx1 == 2

                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataIn.dataType = dataTypeDisc;
    dataIn.dataTenor = dataTenorDisc;
    % important !!
    
    dataIn.dataRate = dataRateDiscSim;
    
    yieldCurveParamsDisc = {};
    
    yieldCurveParamsDisc.floatingLegTenor = '3M';
    yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
    yieldCurveParamsDisc.floatingLegConvention = 'Following';
    yieldCurveParamsDisc.fixingDays = 1;
    
    yieldCurveParamsDisc.fixedLegTenor = '6M';
    yieldCurveParamsDisc.fixedLegDayCounter = '30360';
    yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
    
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];
    yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,dataIn,yieldCurveParamsDisc);
    
    rawData = curveOutDisc(:,1:2);

%     rawData = [0.002739726	0.999959 
%     0.25	0.996135 
%     0.5	0.991903 
%     0.75	0.987432 
%     1	0.982765 
%     2	0.963734 
%     3	0.943126 
%     5	0.897282 
%     10	0.789258 
%     20	0.628672 
%     30	0.503774 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
%simul_delta_test_cms_spread_ra_lgm2f_20180828_newParams0816_yc.m

clc
clear
valueDate = '2018-08-27';
dateH = H_Date(valueDate);
% valueDate = '2018-08-28';

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
volShiftAmount = 0.001;
curveShiftAmount = 0.0001;

rmseTotalSim = zeros(100,1);
modelParamsSim = zeros(100,100);
callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';

% deltaSimulationType = 'DiscParallelDelta';
deltaSimulationType = 'DiscPointByPointDelta';

% vegaSimulationType = 'ParallelVega';
% vegaSimulationType = 'TermVega';
% vegaSimulationType = 'PointByPointVega';
vegaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*16+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
elseif strcmp(vegaSimulationType,'ParallelVega')
    idxMax = 2*1 +1;
elseif strcmp(vegaSimulationType,'TermVega')
    idxMax = 2*8 +1;
elseif strcmp(vegaSimulationType,'PointByPointVega')
    idxMax = 2*8*9+1;
else
    idxMax = 2*1+1;
end

% idxMax = 2*1 +1;
% idxMax = 2*8 +1;
% idxMax = 2*8*9+1;
for idx1=1:idxMax
    %% import and construct a zeroCurve
    dataType= {'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenor = {'1D'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '4Y'
              '5Y'
              '7Y'
              '10Y'
              '12Y'
              '15Y'
              '20Y'
              '25Y'
              '30Y'
    };

    dataRate = {0.0148
    0.0165
    0.01725
    0.017775
    0.01805
    0.018725
    0.019075
    0.019425
    0.019775
    0.020175
    0.020825
    0.0212
    0.02135
    0.02145
    0.02145
    0.02145
    };

    dataRateSim = dataRate;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateSim = dataRate;
        else
            if idx1 == 2

                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRate)
                    dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idx1==1
            dataRateSim = dataRate;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateSim{idx2} = dataRate{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateSim{idx2} = dataRate{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataIn.dataType = dataType;
    dataIn.dataTenor = dataTenor;
    % important !!
    
    dataIn.dataRate = dataRateSim;
    
    yieldCurveParams = {};
    yieldCurveParams.fixedLegTenor = '3M';
    yieldCurveParams.fixedLegDayCounter = 'Act365';
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
4
5
7
10
12
15
20
25
30];
    yieldCurveParams.targetTimeBucket = targetTimeBucket;
    curveOut = YieldCurveBootStrap('KRW',dateH,dataIn,yieldCurveParams);
    
    rawData = curveOut(:,1:2);
%     rawData = [0.002739726	0.999959 
%     0.25	0.995904 
%     0.5	0.991447 
%     0.75	0.986791 
%     1	0.982157 
%     2	0.963331 
%     3	0.944487 
%     4	0.925354 
%     5	0.905952 
%     7	0.868350 
%     10	0.811785 
%     12	0.774859 
%     15	0.725355 
%     20	0.650550 
%     25	0.584638 
%     30	0.525482 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    dataTypeDisc= {'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenorDisc = {'1D'
              '3M'
              '6M'
              '9M'
              '1Y'
              '2Y'
              '3Y'
              '5Y'
              '10Y'
              '20Y'
              '30Y'
    };

    dataRateDisc = {0.014800 
0.015540 
0.016330 
0.016980 
0.017470 
0.018550 
0.019590 
0.021720 
0.023640 
0.023320 
0.023050 
    };

    dataRateDiscSim = dataRateDisc;
    
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if idx1 == 2

                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                end

            elseif idx1 ==3
                for idx2= 1: length(dataRateDisc)
                    dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        if idx1==1
            dataRateDiscSim = dataRateDisc;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                dataRateDiscSim{idx2} = dataRateDisc{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataIn.dataType = dataTypeDisc;
    dataIn.dataTenor = dataTenorDisc;
    % important !!
    
    dataIn.dataRate = dataRateDiscSim;
    
    yieldCurveParamsDisc = {};
    yieldCurveParamsDisc.fixedLegTenor = '6M';
    yieldCurveParamsDisc.fixedLegDayCounter = '30360';
    targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];
    yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,dataIn,yieldCurveParamsDisc);
    
    rawData = curveOutDisc(:,1:2);

%     rawData = [0.002739726	0.999959 
%     0.25	0.996135 
%     0.5	0.991903 
%     0.75	0.987432 
%     1	0.982765 
%     2	0.963734 
%     3	0.943126 
%     5	0.897282 
%     10	0.789258 
%     20	0.628672 
%     30	0.503774 
%     ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    AllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.196088	0.199311	0.202859	0.208283	0.21106	0.211796	0.215756	0.215527	0.215527
    2	0.215955	0.215711	0.212276	0.211615	0.211179	0.209475	0.211285	0.212742	0.212742
    3	0.220167	0.213383	0.207033	0.203721	0.200963	0.196879	0.198009	0.201696	0.201696
    4	0.218895	0.213354	0.206273	0.201317	0.197829	0.193508	0.196126	0.199657	0.199657
    5	0.221935	0.21319	0.205208	0.19885	0.196614	0.192815	0.19678	0.200242	0.200242
    7	0.217022	0.203716	0.198962	0.190091	0.183535	0.183677	0.191818	0.193921	0.193921
    10	0.205276	0.191366	0.187957	0.186262	0.182452	0.182149	0.186734	0.188778	0.188778
    15	0.21679	0.213865	0.196587	0.192188	0.189586	0.188942	0.192672	0.184536	0.184536
    ];

    rawDataSwaptionSim = rawDataSwaption;
    
    if strcmp(vegaSimulationType,'ParallelVega')   
    
%% volatility surface simulation by parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if idx1 == 2

                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
                    end
                end

            elseif idx1 ==3
                for idx2= 2: size(rawDataSwaption,1)
                    for idx3=2:size(rawDataSwaption,2)
                        rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
                    end
                end
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolUp') 
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) + volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'ParallelVolDown')
        for idx2= 2: size(rawDataSwaption,1)
            for idx3=2:size(rawDataSwaption,2)
                rawDataSwaptionSim(idx2,idx3) = rawDataSwaption(idx2,idx3) - volShiftAmount*1.0;
            end
        end
    elseif strcmp(vegaSimulationType,'TermVega')   
    
%% volatility surface simulation by termwise parallel translation for vega

        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) + volShiftAmount*1.0;
                end
                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                for idx3=2:size(rawDataSwaption,2)
                    rawDataSwaptionSim(idx2+1,idx3) = rawDataSwaption(idx2+1,idx3) - volShiftAmount*1.0;
                end
                
            end
        end
    elseif strcmp(vegaSimulationType,'TermVolUp')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) + volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'TermVolDown')
        for idx3=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idx1+1,idx3) = rawDataSwaption(idx1+1,idx3) - volShiftAmount*1.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')   
    
%% volatility surface simulation by point by point vega
         dummyPlus = 1;
        if idx1==1
            rawDataSwaptionSim = rawDataSwaption;
        else
            if mod(idx1,2) == 0
                idx2 = idx1/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) + volShiftAmount*1.0;
                                
            elseif mod(idx1,2) == 1
                idx2 = (idx1-1)/2;
                colIdx = mod(idx2,9);
                if colIdx==0
                    colIdx = 9;
                    dummyPlus = 0;
                end
                rowIdx = floor(idx2/9)+dummyPlus; 
                rawDataSwaptionSim(rowIdx+1,colIdx+1) = rawDataSwaption(rowIdx+1,colIdx+1) - volShiftAmount*1.0;
                
            end
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    AllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

%     %JaeYeol
%     dHTest = [0.060438773	0.585467789
%     0.064928789	0.621766986
%     0.059045192	0.705355899
%     0.070178721	0.6972879
%     0.084746775	0.641430112
%     0.079080439	0.601700141
%     0.1039792	0.504405369
%     0.129262749	0.522031261
%     0.14309303	0.436403222
%     0.167840108	0.341211636
%     0.171134327	0.384814515
%     0.17295042	0.333229804
%     0.205446544	0.253016302
%     0.190785578	0.233852044
%     0.212042588	0.235092969
%     0.208563343	0.261106368
%     0.26724335	0.213909029
%     0.24410813	0.131731127
%     0.292886806	0.110481688
%     0.302159283	0.107800708
%     0.341084552	0.095566454
%     0.353944447	0.072422244
%     0.421105867	0.033562964
%     0.357823112	0.006568462
%     ];

    
    %NewParams0816
    
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

%     %jaeweol
%     AlphaTest = [0.032529518	0.005904921
%     0.021882297	0.0061619
%     0.015498414	0.006184972
%     0.011471555	0.007705108
%     0.008105853	0.008915352
%     0.010349202	0.008306695
%     0.006993193	0.012332999
%     0.000309834	0.020072884
%     ];

 %NewParams0816
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
    % import irModels.*;
    % import targetFunctions.*;

    KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


    tic
    out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    
    rmseTotalSim(idx1) = out.swaptionPriceRmseTotal;
    
    for i=1:length(dH1.quote)
        modelParamsSim(idx1,i) = out.dH1.quote(i);
        modelParamsSim(idx1,i + length(dH1.quote)) = out.dH2.quote(i);
    end
    
    for i=1:length(Alpha1.quote)
        modelParamsSim(idx1,i + 2*length(dH1.quote)) = out.Alpha1.quote(i);
        modelParamsSim(idx1,i + 2*length(dH1.quote)+length(Alpha1.quote)) = out.Alpha2.quote(i);
    end
    
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc

    dH1Calib = KRWLGM.dH1;
    dH2Calib = KRWLGM.dH2;
    Alpha1Calib = KRWLGM.Alpha1;
    Alpha2Calib = KRWLGM.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
    %% LGM model building end
    %% CMS CMSRangeAccrual valuation start
    %RA info
    rangeAccrualParams.payCcy = 'KRW';
    rangeAccrualParams.startDate = H_Date('2017-02-17');
    rangeAccrualParams.maturity = 15;

    % valueDateH = H_Date('2017-04-06');
    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.0362;
    couponFreq = 1;
    tenorL = 10;
    tenorS = 2;
    lower = 0.0;
    upper = 999;
    rangeDays = zeros(15,1);
    %rangeDays(1) = 1;
    % rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


    callYN = ones(15,1);
    callYN(1) = 0;
    callYN(2) = 0;
    callYN(3) = 0;

    %resetDate startDate endDate paymentDate
    couponScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200217	20200217	20210217	20210217
    20210217	20210217	20220217	20220217
    20220217	20220217	20230217	20230217
    20230217	20230217	20240217	20240219
    20240216	20240217	20250217	20250217
    20250217	20250217	20260217	20260219
    20260213	20260217	20270217	20270217
    20270217	20270217	20280217	20280217
    20280217	20280217	20290217	20290219
    20290216	20290217	20300217	20300218
    20300215	20300217	20310217	20310217
    20310217	20310217	20320217	20320217
    ];

    % couponScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20210217
    % 20210217	20210217	20220217	20220217
    % 20220217	20220217	20230217	20230217
    % 20230217	20230217	20240217	20240217
    % 20240217	20240217	20250217	20250217
    % 20250217	20250217	20260217	20260217
    % 20260217	20260217	20270217	20270217
    % 20270217	20270217	20280217	20280217
    % 20280217	20280217	20290217	20290217
    % 20290217	20290217	20300217	20300217
    % 20300217	20300217	20310217	20310217
    % 20310217	20310217	20320217	20320217
    % ];

    %resetDate startDate endDate paymentDate
    callScheduleInt = [20170217	20170217	20180217	20180219
    20180214	20180217	20190217	20190218
    20190215	20190217	20200217	20200217
    20200210	20200217	20210217	20200217
    20210208	20210217	20220217	20210217
    20220210	20220217	20230217	20220217
    20230210	20230217	20240217	20230217
    20240208	20240217	20250217	20240219
    20250210	20250217	20260217	20250217
    20260209	20260217	20270217	20260219
    20270210	20270217	20280217	20270217
    20280210	20280217	20290217	20280217
    20290207	20290217	20300217	20290219
    20300211	20300217	20310217	20300218
    20310210	20310217	20320217	20310217
    ];

    % callScheduleInt = [20170217	20170217	20180217	20180217
    % 20180217	20180217	20190217	20190217
    % 20190217	20190217	20200217	20200217
    % 20200217	20200217	20210217	20200217
    % 20210217	20210217	20220217	20210217
    % 20220217	20220217	20230217	20220217
    % 20230217	20230217	20240217	20230217
    % 20240217	20240217	20250217	20240217
    % 20250217	20250217	20260217	20250217
    % 20260217	20260217	20270217	20260217
    % 20270217	20270217	20280217	20270217
    % 20280217	20280217	20290217	20280217
    % 20290217	20290217	20300217	20290217
    % 20300217	20300217	20310217	20300217
    % 20310217	20310217	20320217	20310217
    % ];

    manualScheduleYN = 1;

    for i=1:length(rangeDays)
        if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end

    rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                    'tenorS','lower','upper','rangeDays','callYN', ...
                                    'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                            {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                            upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;

    rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    pricingOutMC = pricer.computePriceMCCallable(valueDateH);
    
    
    
    callablePrice(idx1) = pricingOutMC.callable.npv;
    nonCallPrice(idx1) = pricingOutMC.nonCall.npv;
%     closedPrice(idx) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx1,idx3) = pricingOutMC.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
end

aaa =1.0;


In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
